<a href="https://colab.research.google.com/github/tomknightatl/usccb-parish-extraction/blob/main/03_Extract_Parish_Data%20(older%20but%20working).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================================================================
# CELL 1: Install and Import Dependencies (FIXED)
# =============================================================================

# Install additional dependencies for the enhanced system
!pip install supabase dataclasses-json beautifulsoup4 selenium webdriver-manager tenacity

# Complete imports including missing ones
import os
import time
import json
import random
import sqlite3
import pandas as pd
import subprocess  # FIXED: Added for Chrome installation
import re  # FIXED: Added for pattern detection
from datetime import datetime
from dataclasses import dataclass, asdict
from enum import Enum
from typing import List, Dict, Optional, Any
from urllib.parse import urljoin, urlparse

# Web scraping
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Your existing Supabase and AI imports
from google.colab import userdata
from supabase import create_client, Client

print("✅ All dependencies installed and imported successfully!")

✅ All dependencies installed and imported successfully!


In [ ]:
# =============================================================================
# CELL 2: Configuration (Reuse your existing setup)
# =============================================================================

# Reuse your existing configuration logic
print("=== ENHANCED PARISH EXTRACTOR CONFIGURATION ===")

# GitHub and database setup (copy from your existing notebooks)
GITHUB_REPO = 'USCCB'
GITHUB_USERNAME = userdata.get('GitHubUserforUSCCB')
GITHUB_PAT = userdata.get('GitHubPATforUSCCB')

# Supabase configuration (copy from your existing setup)
SUPABASE_URL = userdata.get('SUPABASE_URL')
SUPABASE_KEY = userdata.get('SUPABASE_KEY')

if SUPABASE_URL and SUPABASE_KEY:
    supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
    print("✅ Supabase client initialized")
else:
    print("❌ Supabase credentials not found")
    supabase = None

# Processing configuration
MAX_DIOCESES_TO_PROCESS = 1  # Start small for testing
ENABLE_PATTERN_DETECTION = True
SAVE_DETAILED_LOGS = True

print(f"📊 Will process {MAX_DIOCESES_TO_PROCESS} dioceses with pattern detection")

=== ENHANCED PARISH EXTRACTOR CONFIGURATION ===
✅ Supabase client initialized
📊 Will process 1 dioceses with pattern detection


In [ ]:
# =============================================================================
# CELL 2.5: Chrome Installation for Google Colab (FIXED)
# =============================================================================

import subprocess  # This was missing!
import os

def ensure_chrome_installed():
    """Ensures Chrome is installed in the Colab environment."""
    try:
        # Check if Chrome is already available
        result = subprocess.run(['which', 'google-chrome'], capture_output=True, text=True)
        if result.returncode == 0:
            print("✅ Chrome is already installed and available.")
            return True

        print("🔧 Chrome not found. Installing Chrome for Selenium...")

        # Install Chrome
        os.system('apt-get update > /dev/null 2>&1')
        os.system('wget -q -O - https://dl.google.com/linux/linux_signing_key.pub | apt-key add - > /dev/null 2>&1')
        os.system('echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" > /etc/apt/sources.list.d/google-chrome.list')
        os.system('apt-get update > /dev/null 2>&1')
        os.system('apt-get install -y google-chrome-stable > /dev/null 2>&1')

        # Verify installation
        result = subprocess.run(['google-chrome', '--version'], capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✅ Chrome installed successfully: {result.stdout.strip()}")
            return True
        else:
            print("❌ Chrome installation may have failed.")
            return False

    except Exception as e:
        print(f"❌ Error during Chrome installation: {e}")
        return False

# Run the installation check
print("🔧 Checking Chrome installation...")
chrome_ready = ensure_chrome_installed()

if chrome_ready:
    print("🚀 Ready to proceed with Selenium operations!")
else:
    print("⚠️ You may need to restart the runtime if Chrome installation failed.")

# Test Chrome installation
if chrome_ready:
    try:
        result = subprocess.run(['google-chrome', '--version'], capture_output=True, text=True)
        print(f"📋 Chrome version: {result.stdout.strip()}")
    except Exception as e:
        print(f"⚠️ Chrome test failed: {e}")
        chrome_ready = False

print(f"Final Chrome status: {'✅ Ready' if chrome_ready else '❌ Not Ready'}")

🔧 Checking Chrome installation...
✅ Chrome is already installed and available.
🚀 Ready to proceed with Selenium operations!
📋 Chrome version: Google Chrome 137.0.7151.55
Final Chrome status: ✅ Ready


In [ ]:
# =============================================================================
# CELL 2.6: Driver Setup Function (MISSING - NOW ADDED)
# =============================================================================

def setup_enhanced_driver():
    """Setup Chrome driver with enhanced options for parish extraction"""

    try:
        print("🔧 Setting up Chrome driver...")

        # Chrome options
        chrome_options = Options()
        chrome_options.add_argument('--headless')  # Run in background
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('--disable-gpu')
        chrome_options.add_argument('--window-size=1920,1080')
        chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
        chrome_options.add_argument('--disable-blink-features=AutomationControlled')
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)

        # Setup driver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)

        # Additional setup to avoid detection
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

        # Set timeouts
        driver.implicitly_wait(10)
        driver.set_page_load_timeout(30)

        print("✅ Chrome driver setup complete")
        return driver

    except Exception as e:
        print(f"❌ Failed to setup driver: {e}")
        raise

print("✅ Driver setup function added")

✅ Driver setup function added


In [ ]:
# =============================================================================
# CELL 3: Enhanced Pattern Detection Classes (UPDATED for Parish Finder)
# =============================================================================

import re  # This was also missing!

class DiocesePlatform(Enum):
    SQUARESPACE = "squarespace"
    WORDPRESS = "wordpress"
    DRUPAL = "drupal"
    CUSTOM_CMS = "custom"
    STATIC_HTML = "static"
    ECATHOLIC = "ecatholic"  # NEW: Added for sites like Tulsa and Parma
    DIOCESAN_CUSTOM = "diocesan_custom"  # NEW: For custom diocese sites like Salt Lake City
    UNKNOWN = "unknown"

class ParishListingType(Enum):
    INTERACTIVE_MAP = "interactive_map"
    STATIC_TABLE = "static_table"
    CARD_GRID = "card_grid"
    SIMPLE_LIST = "simple_list"
    PAGINATED_LIST = "paginated_list"
    SEARCHABLE_DIRECTORY = "searchable_directory"
    PARISH_FINDER = "parish_finder"  # NEW: Added for interactive parish finders
    DIOCESE_CARD_LAYOUT = "diocese_card_layout"  # NEW: For Salt Lake City style layouts
    PDF_DIRECTORY = "pdf_directory"
    UNKNOWN = "unknown"

@dataclass
class ParishData:
    name: str
    address: Optional[str] = None
    city: Optional[str] = None
    state: Optional[str] = None
    zip_code: Optional[str] = None
    phone: Optional[str] = None
    website: Optional[str] = None
    latitude: Optional[float] = None
    longitude: Optional[float] = None
    pastor: Optional[str] = None
    mass_times: Optional[str] = None
    # NEW: Enhanced fields for detailed extraction
    street_address: Optional[str] = None
    full_address: Optional[str] = None
    parish_detail_url: Optional[str] = None
    clergy_info: Optional[str] = None
    service_times: Optional[str] = None
    # Metadata fields
    confidence_score: float = 0.5
    extraction_method: str = "unknown"
    diocese_url: Optional[str] = None
    parish_directory_url: Optional[str] = None
    detail_extraction_success: bool = False
    detail_extraction_error: Optional[str] = None

@dataclass
class DioceseSitePattern:
    platform: DiocesePlatform
    listing_type: ParishListingType
    confidence_score: float
    extraction_method: str
    specific_selectors: Dict[str, str]
    javascript_required: bool
    pagination_pattern: Optional[str] = None
    notes: str = ""

class PatternDetector:
    """Detects patterns in diocese websites for targeted extraction"""

    def detect_pattern(self, html_content: str, url: str) -> DioceseSitePattern:
        """Analyze website content and detect the best extraction pattern"""
        soup = BeautifulSoup(html_content, 'html.parser')
        html_lower = html_content.lower()

        # Platform detection
        platform = self._detect_platform(html_lower, url)

        # Listing type detection
        listing_type = self._detect_listing_type(html_lower, soup, url)

        # JavaScript requirement
        js_required = self._requires_javascript(html_lower)

        # Determine extraction method and confidence
        extraction_method, confidence, selectors, notes = self._determine_extraction_strategy(
            platform, listing_type, soup, html_lower, url
        )

        return DioceseSitePattern(
            platform=platform,
            listing_type=listing_type,
            confidence_score=confidence,
            extraction_method=extraction_method,
            specific_selectors=selectors,
            javascript_required=js_required,
            notes=notes
        )

    def _detect_platform(self, html_lower: str, url: str) -> DiocesePlatform:
        """Detect CMS/platform"""
        if 'ecatholic.com' in url or 'ecatholic' in html_lower:
            return DiocesePlatform.ECATHOLIC
        elif 'squarespace' in html_lower:
            return DiocesePlatform.SQUARESPACE
        elif 'wp-content' in html_lower or 'wordpress' in html_lower:
            return DiocesePlatform.WORDPRESS
        elif 'drupal' in html_lower:
            return DiocesePlatform.DRUPAL
        elif 'dioslc.org' in url or 'utahcatholicdiocese.org' in url:
            return DiocesePlatform.DIOCESAN_CUSTOM
        else:
            return DiocesePlatform.CUSTOM_CMS

    def _detect_listing_type(self, html_lower: str, soup: BeautifulSoup, url: str) -> ParishListingType:
        """Detect how parishes are listed (UPDATED for better Parish Finder detection)"""

        # Check for Salt Lake City style card layout
        if ('col-lg location' in html_lower and 'card-title' in html_lower and
            'dioslc.org' in url):
            return ParishListingType.DIOCESE_CARD_LAYOUT

        # Enhanced Parish Finder detection for eCatholic sites like Parma
        parish_finder_indicators = [
            'parishfinder' in url.lower(),
            'parish-finder' in url.lower(),
            'finderCore' in html_lower,
            'finder.js' in html_lower,
            'parish finder' in html_lower,
            # NEW: Additional indicators for Parma-style finder
            'li.site' in html_lower and 'siteInfo' in html_lower,
            'finderBar' in html_lower,
            'categories' in html_lower and 'sites' in html_lower and 'parishes' in html_lower,
            # Look for the specific structure
            soup.find('ul', id='categories'),
            soup.find('div', id='finderCore'),
            soup.find('li', class_='site')
        ]

        if any(parish_finder_indicators):
            return ParishListingType.PARISH_FINDER

        # Interactive map indicators
        map_indicators = ['leaflet', 'google.maps', 'mapbox', 'parish-map', 'interactive']
        if any(indicator in html_lower for indicator in map_indicators):
            return ParishListingType.INTERACTIVE_MAP

        # Table indicators
        if soup.find('table') and ('parish' in html_lower or 'church' in html_lower):
            return ParishListingType.STATIC_TABLE

        # Card/grid layout (generic)
        if soup.find_all(class_=re.compile(r'(card|grid|parish-item)', re.I)):
            return ParishListingType.CARD_GRID

        # Pagination
        if any(word in html_lower for word in ['pagination', 'page-numbers', 'next-page']):
            return ParishListingType.PAGINATED_LIST

        return ParishListingType.SIMPLE_LIST

    def _requires_javascript(self, html_lower: str) -> bool:
        """Check if JavaScript is required"""
        js_indicators = ['react', 'angular', 'vue', 'leaflet', 'google.maps', 'ajax', 'finder.js']
        return any(indicator in html_lower for indicator in js_indicators)

    def _determine_extraction_strategy(self, platform, listing_type, soup, html_lower, url):
        """Determine the best extraction strategy (UPDATED for Parish Finder)"""

        if listing_type == ParishListingType.DIOCESE_CARD_LAYOUT:
            return (
                "diocese_card_extraction_with_details",
                0.95,
                {
                    "parish_cards": ".col-lg.location",
                    "parish_name": ".card-title",
                    "parish_city": ".card-body",
                    "parish_link": "a.card"
                },
                "Diocese card layout detected - specialized extraction for Salt Lake City style with detail page navigation"
            )

        elif listing_type == ParishListingType.PARISH_FINDER:
            return (
                "parish_finder_extraction",
                0.95,
                {
                    "parish_list": "li.site, .site",
                    "parish_name": ".name",
                    "parish_city": ".city",
                    "parish_info": ".siteInfo",
                    "parish_details": ".details",
                    "categories": "#categories",
                    "sites_list": ".sites"
                },
                "Parish finder interface detected - specialized extraction for eCatholic-style interactive directory"
            )

        elif listing_type == ParishListingType.INTERACTIVE_MAP:
            return (
                "interactive_map_extraction",
                0.9,
                {"map_container": "#map, .map-container, .parish-map"},
                "Interactive map detected - will extract from JS data and markers"
            )

        elif listing_type == ParishListingType.STATIC_TABLE:
            return (
                "table_extraction",
                0.95,
                {"table": "table", "rows": "tr:not(:first-child)"},
                "HTML table detected - most reliable extraction method"
            )

        elif platform == DiocesePlatform.SQUARESPACE:
            return (
                "squarespace_extraction",
                0.8,
                {"items": ".summary-item, .parish-item", "title": ".summary-title"},
                "SquareSpace platform - using platform-specific selectors"
            )

        else:
            return (
                "generic_extraction",
                0.4,
                {"containers": "[class*='parish'], [class*='church']"},
                "Using generic extraction patterns"
            )

print("✅ Enhanced pattern detection classes loaded with improved Parish Finder support")

✅ Enhanced pattern detection classes loaded with improved Parish Finder support


In [ ]:
# =============================================================================
# CELL 4: Enhanced DiocesesCardExtractor with Detail Page Navigation (UPDATED)
# =============================================================================

class EnhancedDiocesesCardExtractor(BaseExtractor):
    """Enhanced extractor that clicks on each parish card to get detailed information"""

    def __init__(self, pattern: DioceseSitePattern):
        super().__init__(pattern)
        self.detail_extraction_count = 0
        self.detail_extraction_errors = 0

    def extract(self, driver, soup: BeautifulSoup, url: str) -> List[ParishData]:
        parishes = []

        try:
            print("    📍 Enhanced diocese card layout detected - extracting with detail pages")

            # Find all parish cards using the specific Salt Lake City structure
            parish_cards = soup.find_all('div', class_='col-lg location')
            print(f"    📊 Found {len(parish_cards)} parish cards")

            for i, card in enumerate(parish_cards, 1):
                try:
                    print(f"    🔄 Processing parish {i}/{len(parish_cards)}")
                    parish_data = self._extract_parish_from_card_with_details(card, url, driver, i)
                    if parish_data:
                        parishes.append(parish_data)
                        if parish_data.detail_extraction_success:
                            self.detail_extraction_count += 1
                        else:
                            self.detail_extraction_errors += 1
                except Exception as e:
                    print(f"    ⚠️ Error extracting from card {i}: {str(e)[:100]}...")
                    self.detail_extraction_errors += 1
                    continue

            print(f"    📊 Summary: {self.detail_extraction_count} detailed extractions successful, {self.detail_extraction_errors} failed")

        except Exception as e:
            print(f"    ⚠️ Enhanced diocese card extraction error: {str(e)[:100]}...")

        return parishes

    def _extract_parish_from_card_with_details(self, card, base_url: str, driver, card_number: int) -> Optional[ParishData]:
        """Extract parish data from a single card and navigate to detail page"""
        try:
            # Step 1: Extract basic information from the card
            card_link = card.find('a', class_='card')
            if not card_link:
                return None

            # Extract parish name from card title
            title_elem = card_link.find('h4', class_='card-title')
            if not title_elem:
                return None

            name = self.clean_text(title_elem.get_text())
            if not name or len(name) < 3:
                return None

            # Skip non-parish entries
            skip_terms = [
                'no parish registration', 'contact', 'chancery', 'pastoral center',
                'tv mass', 'directory', 'finder', 'diocese', 'bishop', 'office'
            ]
            if any(term in name.lower() for term in skip_terms):
                return None

            # Extract city from card body
            card_body = card_link.find('div', class_='card-body')
            city = None
            state = None
            if card_body:
                body_text = card_body.get_text()
                lines = [line.strip() for line in body_text.split('\n') if line.strip()]

                # The city is usually the second line (after the parish name)
                if len(lines) >= 2:
                    city_line = lines[1]
                    if city_line and not city_line.startswith('Learn More'):
                        city = self.clean_text(city_line)

            # Extract parish detail URL
            parish_detail_url = None
            href = card_link.get('href')
            if href:
                if href.startswith('/'):
                    from urllib.parse import urljoin
                    parish_detail_url = urljoin(base_url, href)
                else:
                    parish_detail_url = href

            # Extract state from city if present (format: "City, ST")
            if city and ', ' in city:
                city_parts = city.split(', ')
                if len(city_parts) == 2:
                    city = city_parts[0].strip()
                    state = city_parts[1].strip()

            # Step 2: Navigate to detail page and extract additional information
            detailed_info = self._extract_details_from_parish_page(driver, parish_detail_url, name)

            # Step 3: Create comprehensive parish data object
            parish_data = ParishData(
                name=name,
                city=city,
                state=state,
                parish_detail_url=parish_detail_url,
                confidence_score=0.9,
                extraction_method="enhanced_diocese_card_extraction"
            )

            # Add detailed information if extraction was successful
            if detailed_info['success']:
                parish_data.street_address = detailed_info.get('street_address')
                parish_data.full_address = detailed_info.get('full_address')
                parish_data.zip_code = detailed_info.get('zip_code')
                parish_data.phone = detailed_info.get('phone')
                parish_data.website = detailed_info.get('website')
                parish_data.clergy_info = detailed_info.get('clergy_info')
                parish_data.service_times = detailed_info.get('service_times')
                parish_data.detail_extraction_success = True
                parish_data.confidence_score = 0.95  # Higher confidence with detailed info
                print(f"      ✅ {name}: Complete details extracted")
            else:
                parish_data.detail_extraction_success = False
                parish_data.detail_extraction_error = detailed_info.get('error')
                print(f"      ⚠️ {name}: Basic info only - {detailed_info.get('error', 'Unknown error')}")

            return parish_data

        except Exception as e:
            print(f"    ⚠️ Error parsing card {card_number}: {str(e)[:50]}...")
            return None

    def _extract_details_from_parish_page(self, driver, parish_url: str, parish_name: str) -> Dict:
        """Navigate to parish detail page and extract detailed information"""

        if not parish_url:
            return {'success': False, 'error': 'No detail URL available'}

        try:
            print(f"      🔗 Navigating to: {parish_url}")

            # Navigate to the parish detail page
            driver.get(parish_url)
            time.sleep(2)  # Wait for page to load

            # Get the page source and parse it
            detail_html = driver.page_source
            detail_soup = BeautifulSoup(detail_html, 'html.parser')

            # Initialize result dictionary
            result = {
                'success': True,
                'street_address': None,
                'full_address': None,
                'zip_code': None,
                'phone': None,
                'website': None,
                'clergy_info': None,
                'service_times': None
            }

            # Extract contact information from the detail page
            self._extract_contact_info(detail_soup, result)
            self._extract_service_times(detail_soup, result)
            self._extract_clergy_info(detail_soup, result)

            return result

        except Exception as e:
            error_msg = f"Failed to extract details: {str(e)[:100]}"
            print(f"      ❌ {parish_name}: {error_msg}")
            return {'success': False, 'error': error_msg}

    def _extract_contact_info(self, soup: BeautifulSoup, result: Dict):
        """Extract contact information from parish detail page"""
        try:
            # Look for contact info section
            contact_sections = soup.find_all(['div', 'section'], class_=re.compile(r'contact', re.I))

            # Also look for FA icons which often indicate contact info
            fa_ul_sections = soup.find_all('ul', class_='fa-ul')

            all_contact_sections = contact_sections + fa_ul_sections

            for section in all_contact_sections:
                text_content = section.get_text()

                # Extract phone number
                if not result['phone']:
                    phone_links = section.find_all('a', href=re.compile(r'^tel:'))
                    if phone_links:
                        phone = phone_links[0].get_text().strip()
                        result['phone'] = self.clean_text(phone)
                    else:
                        # Look for phone patterns in text
                        phone_match = re.search(r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', text_content)
                        if phone_match:
                            result['phone'] = phone_match.group()

                # Extract website
                if not result['website']:
                    website_links = section.find_all('a', href=re.compile(r'^http'))
                    for link in website_links:
                        href = link.get('href', '')
                        # Skip social media and diocese links
                        if not any(skip in href.lower() for skip in ['facebook', 'twitter', 'instagram', 'dioslc.org']):
                            result['website'] = href
                            break

                # Extract address
                if not result['full_address']:
                    # Look for address patterns
                    address_lines = []
                    list_items = section.find_all('li')

                    for li in list_items:
                        li_text = li.get_text().strip()
                        # Check if this looks like an address (contains numbers and common address words)
                        if re.search(r'\d+.*(?:street|st|avenue|ave|road|rd|drive|dr|way|lane|ln|boulevard|blvd)', li_text, re.I):
                            address_lines.append(li_text)
                        elif re.search(r'\d+\s+[A-Za-z]', li_text) and ',' in li_text:
                            # Format like "331 East South Temple Street, Salt Lake City, UT 84111"
                            address_lines.append(li_text)

                    if address_lines:
                        full_address = address_lines[0]
                        result['full_address'] = full_address

                        # Try to parse street address and zip code
                        self._parse_address_components(full_address, result)

        except Exception as e:
            print(f"        ⚠️ Error extracting contact info: {str(e)[:50]}")

    def _parse_address_components(self, full_address: str, result: Dict):
        """Parse full address into street address and zip code"""
        try:
            # Extract zip code (5 digits, possibly followed by 4 more)
            zip_match = re.search(r'\b(\d{5}(?:-\d{4})?)\b', full_address)
            if zip_match:
                result['zip_code'] = zip_match.group(1)

            # Extract street address (everything before the first comma, or before city/state)
            address_parts = full_address.split(',')
            if len(address_parts) > 0:
                potential_street = address_parts[0].strip()
                # Make sure it looks like a street address
                if re.search(r'\d+', potential_street):
                    result['street_address'] = potential_street

        except Exception as e:
            print(f"        ⚠️ Error parsing address: {str(e)[:30]}")

    def _extract_service_times(self, soup: BeautifulSoup, result: Dict):
        """Extract service times from parish detail page"""
        try:
            # Look for service times section
            service_sections = soup.find_all(['div', 'section'],
                                           string=re.compile(r'service.*times|mass.*times|masses|schedule', re.I))

            # Also look for h3 headers that might indicate service times
            service_headers = soup.find_all(['h3', 'h4'],
                                          string=re.compile(r'service.*times|mass.*times|masses|schedule', re.I))

            # Look for lists that might contain service information
            service_lists = []
            for header in service_headers:
                next_sibling = header.find_next_sibling(['ul', 'div'])
                if next_sibling:
                    service_lists.append(next_sibling)

            all_service_sections = service_sections + service_lists

            for section in all_service_sections:
                if section:
                    service_text = section.get_text()
                    # Extract meaningful service time information
                    lines = [line.strip() for line in service_text.split('\n') if line.strip()]
                    # Filter out very short lines and focus on schedule information
                    schedule_lines = [line for line in lines if len(line) > 10 and
                                    any(keyword in line.lower() for keyword in
                                        ['sunday', 'saturday', 'daily', 'mass', 'service', 'am', 'pm'])]

                    if schedule_lines:
                        result['service_times'] = '; '.join(schedule_lines[:5])  # Limit to first 5 lines
                        break

        except Exception as e:
            print(f"        ⚠️ Error extracting service times: {str(e)[:50]}")

    def _extract_clergy_info(self, soup: BeautifulSoup, result: Dict):
        """Extract clergy information from parish detail page"""
        try:
            # Look for clergy section
            clergy_sections = soup.find_all(['div', 'section'], class_=re.compile(r'clergy|pastor|priest', re.I))

            # Also look for directory cards that might contain clergy info
            directory_cards = soup.find_all(['div'], class_=re.compile(r'directory|card', re.I))

            all_clergy_sections = clergy_sections + directory_cards

            clergy_info = []
            for section in all_clergy_sections:
                # Look for names and titles
                titles = section.find_all(['h4', 'h5'], class_=re.compile(r'title|name', re.I))
                for title in titles:
                    title_text = title.get_text().strip()
                    # Check if this looks like a clergy title
                    if any(clergy_word in title_text.lower() for clergy_word in
                           ['reverend', 'father', 'pastor', 'deacon', 'rev.', 'fr.', 'dcn.']):

                        # Get associated role/description
                        role_elem = title.find_next_sibling(['p', 'div'])
                        role_text = role_elem.get_text().strip() if role_elem else ""

                        if role_text:
                            clergy_info.append(f"{title_text}: {role_text}")
                        else:
                            clergy_info.append(title_text)

            if clergy_info:
                result['clergy_info'] = '; '.join(clergy_info[:3])  # Limit to first 3 clergy members

        except Exception as e:
            print(f"        ⚠️ Error extracting clergy info: {str(e)[:50]}")

# Update the original DiocesesCardExtractor to use the enhanced version
DiocesesCardExtractor = EnhancedDiocesesCardExtractor

print("✅ Enhanced DiocesesCardExtractor loaded with detail page navigation")

✅ Enhanced DiocesesCardExtractor loaded with detail page navigation


In [ ]:
# =============================================================================
# CELL 4.2: Base Extractor Classes and Parish Finder Implementation (ADDED)
# =============================================================================

class BaseExtractor:
    """Base class for parish extractors"""

    def __init__(self, pattern: DioceseSitePattern):
        self.pattern = pattern

    def extract(self, driver, soup: BeautifulSoup, url: str) -> List[ParishData]:
        """Extract parishes from the given page"""
        raise NotImplementedError

    def clean_text(self, text: str) -> str:
        """Clean and normalize text"""
        if not text:
            return ""
        return ' '.join(text.strip().split())

    def extract_phone(self, text: str) -> Optional[str]:
        """Extract phone number from text"""
        if not text:
            return None

        # Look for phone patterns
        phone_pattern = r'\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}'
        match = re.search(phone_pattern, text)
        if match:
            return match.group()
        return None

    def extract_email(self, text: str) -> Optional[str]:
        """Extract email from text"""
        if not text:
            return None

        email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        match = re.search(email_pattern, text)
        if match:
            return match.group()
        return None

class ParishFinderExtractor(BaseExtractor):
    """Extractor for eCatholic Parish Finder interfaces (like Parma)"""

    def extract(self, driver, soup: BeautifulSoup, url: str) -> List[ParishData]:
        parishes = []

        try:
            print("    📍 Parish finder interface detected")

            # Try different selectors for parish items
            parish_selectors = [
                "li.site",
                ".site",
                "li[data-latitude]",
                ".parish-item",
                "[class*='parish']"
            ]

            parish_elements = []
            for selector in parish_selectors:
                elements = soup.find_all(class_=lambda x: x and 'site' in x) if selector == "li.site" else soup.select(selector)
                if elements:
                    parish_elements = elements
                    print(f"    📊 Found {len(parish_elements)} parish elements using {selector}")
                    break

            if not parish_elements:
                print("    ⚠️ No parish elements found")
                return parishes

            for i, element in enumerate(parish_elements, 1):
                try:
                    parish_data = self._extract_parish_from_finder_element(element, url, i)
                    if parish_data:
                        parishes.append(parish_data)
                        print(f"      ✅ Extracted: {parish_data.name}")
                    else:
                        print(f"      ⚠️ Skipped element {i}: No valid parish data")

                except Exception as e:
                    print(f"      ❌ Error processing element {i}: {str(e)[:50]}...")
                    continue

            print(f"    📊 Successfully extracted {len(parishes)} parishes from parish finder")

        except Exception as e:
            print(f"    ❌ Parish finder extraction error: {str(e)[:100]}...")

        return parishes

    def _extract_parish_from_finder_element(self, element, base_url: str, element_num: int) -> Optional[ParishData]:
        """Extract parish data from a single parish finder element"""
        try:
            # Extract parish name
            name = None
            name_selectors = ['.name', '.parish-name', 'h3', 'h4', '.title']

            for selector in name_selectors:
                name_elem = element.select_one(selector)
                if name_elem:
                    name = self.clean_text(name_elem.get_text())
                    break

            # If no name found using selectors, try getting the first significant text
            if not name:
                # Look for div with class="name" specifically
                name_div = element.find('div', class_='name')
                if name_div:
                    name = self.clean_text(name_div.get_text())

            if not name or len(name) < 3:
                return None

            # Skip non-parish entries
            skip_terms = [
                'no parish registration', 'contact', 'chancery', 'pastoral center',
                'tv mass', 'directory', 'finder', 'diocese', 'bishop', 'office',
                'search', 'filter', 'map'
            ]
            if any(term in name.lower() for term in skip_terms):
                return None

            # Extract city
            city = None
            city_selectors = ['.city', '.location', '.parish-city']

            for selector in city_selectors:
                city_elem = element.select_one(selector)
                if city_elem:
                    city = self.clean_text(city_elem.get_text())
                    break

            # If no city found using selectors, try getting the city div
            if not city:
                city_div = element.find('div', class_='city')
                if city_div:
                    city = self.clean_text(city_div.get_text())

            # Extract detailed information from siteInfo if available
            site_info = element.find('div', class_='siteInfo')
            if site_info:
                return self._extract_from_site_info(name, city, site_info, element, base_url)

            # Extract address from title or address elements
            address = None
            address_selectors = ['.address', '.street-address', '.location']

            for selector in address_selectors:
                addr_elem = element.select_one(selector)
                if addr_elem:
                    address = self.clean_text(addr_elem.get_text())
                    if address and len(address) > 5:
                        break

            # Extract phone
            phone = None
            phone_selectors = ['.phone', '.telephone', 'a[href^="tel:"]']

            for selector in phone_selectors:
                phone_elem = element.select_one(selector)
                if phone_elem:
                    phone_text = phone_elem.get('href') if phone_elem.get('href', '').startswith('tel:') else phone_elem.get_text()
                    phone = self.extract_phone(phone_text)
                    if phone:
                        break

            # Extract website
            website = None
            website_selectors = ['a[href^="http"]', '.website', '.url']

            for selector in website_selectors:
                web_elem = element.select_one(selector)
                if web_elem:
                    href = web_elem.get('href', '')
                    if href.startswith('http') and not any(skip in href.lower() for skip in ['facebook', 'twitter', 'instagram']):
                        website = href
                        break

            # Extract coordinates if available
            latitude = None
            longitude = None

            if element.get('data-latitude'):
                try:
                    latitude = float(element.get('data-latitude'))
                except (ValueError, TypeError):
                    pass

            if element.get('data-longitude'):
                try:
                    longitude = float(element.get('data-longitude'))
                except (ValueError, TypeError):
                    pass

            return ParishData(
                name=name,
                city=city,
                address=address,
                phone=phone,
                website=website,
                latitude=latitude,
                longitude=longitude,
                confidence_score=0.8,
                extraction_method="parish_finder_extraction"
            )

        except Exception as e:
            print(f"        ⚠️ Error parsing parish element {element_num}: {str(e)[:50]}...")
            return None

    def _extract_from_site_info(self, name: str, city: str, site_info, element, base_url: str) -> Optional[ParishData]:
        """Extract detailed information from siteInfo section"""
        try:
            # Get the main content
            main_section = site_info.find('div', class_='main')
            if not main_section:
                return None

            # Extract address from title section
            title_section = main_section.find('div', class_='title')
            address = None
            phone = None
            email = None

            if title_section:
                # Look for address
                address_div = title_section.find('div', class_='address')
                if address_div:
                    address = self.clean_text(address_div.get_text())

                # Look for phone in phoneFaxHolder
                phone_holder = title_section.find('div', class_='phoneFaxHolder')
                if phone_holder:
                    phone_span = phone_holder.find('span', class_='phone')
                    if phone_span:
                        phone = self.clean_text(phone_span.get_text())

                # Look for email
                email_link = title_section.find('a', class_='email')
                if email_link:
                    email = self.extract_email(email_link.get_text())

            # Extract website from linkContainer
            website = None
            link_container = main_section.find('div', class_='linkContainer')
            if link_container:
                url_link = link_container.find('a', class_='urlLink')
                if url_link:
                    website = url_link.get('href')

            # Extract coordinates from the main element
            latitude = None
            longitude = None

            if element.get('data-latitude'):
                try:
                    latitude = float(element.get('data-latitude'))
                except (ValueError, TypeError):
                    pass

            if element.get('data-longitude'):
                try:
                    longitude = float(element.get('data-longitude'))
                except (ValueError, TypeError):
                    pass

            # Extract detailed info from extendedInfo if available
            clergy_info = None
            service_times = None

            extended_info = site_info.find('div', class_='extendedInfo')
            if extended_info:
                details_div = extended_info.find('div', class_='details')
                if details_div:
                    details_text = details_div.get_text()
                    # This typically contains clergy information
                    if details_text and len(details_text.strip()) > 10:
                        clergy_info = self.clean_text(details_text)

            return ParishData(
                name=name,
                city=city,
                address=address,
                phone=phone,
                website=website,
                latitude=latitude,
                longitude=longitude,
                clergy_info=clergy_info,
                service_times=service_times,
                confidence_score=0.9,
                extraction_method="parish_finder_detailed_extraction",
                detail_extraction_success=True if (address or phone or website or clergy_info) else False
            )

        except Exception as e:
            print(f"        ⚠️ Error extracting from siteInfo: {str(e)[:50]}...")
            return None

class TableExtractor(BaseExtractor):
    """Extractor for HTML table-based parish listings"""

    def extract(self, driver, soup: BeautifulSoup, url: str) -> List[ParishData]:
        parishes = []

        try:
            tables = soup.find_all('table')

            for table in tables:
                table_text = table.get_text().lower()
                if any(keyword in table_text for keyword in ['parish', 'church', 'name', 'address']):
                    rows = table.find_all('tr')[1:]  # Skip header row

                    for row in rows:
                        cells = row.find_all(['td', 'th'])
                        if len(cells) >= 1:
                            parish_data = self._extract_parish_from_table_row(cells, url)
                            if parish_data:
                                parishes.append(parish_data)

        except Exception as e:
            print(f"    ⚠️ Table extraction error: {str(e)[:100]}...")

        return parishes

    def _extract_parish_from_table_row(self, cells, base_url: str) -> Optional[ParishData]:
        """Extract parish data from table row cells"""
        if not cells:
            return None

        # First cell usually contains the name
        name = self.clean_text(cells[0].get_text())
        if not name or len(name) < 3:
            return None

        # Extract other information from remaining cells
        address = None
        city = None
        phone = None
        website = None

        for i, cell in enumerate(cells[1:], 1):
            cell_text = cell.get_text()

            # Check for phone pattern
            if self.extract_phone(cell_text):
                phone = self.extract_phone(cell_text)

            # Check for address pattern (contains numbers and street words)
            elif re.search(r'\d+.*(?:street|st|avenue|ave|road|rd|drive|dr)', cell_text, re.I):
                address = self.clean_text(cell_text)

            # Check for website
            link = cell.find('a')
            if link and link.get('href', '').startswith('http'):
                website = link.get('href')

            # If it looks like a city (short text, no numbers)
            elif len(cell_text.strip()) < 30 and not re.search(r'\d', cell_text) and len(cell_text.strip()) > 2:
                city = self.clean_text(cell_text)

        return ParishData(
            name=name,
            address=address,
            city=city,
            phone=phone,
            website=website,
            confidence_score=0.85,
            extraction_method="table_extraction"
        )

class ImprovedGenericExtractor(BaseExtractor):
    """Improved generic extractor as fallback"""

    def extract(self, driver, soup: BeautifulSoup, url: str) -> List[ParishData]:
        parishes = []

        try:
            print("    🔍 Using improved generic extraction...")

            # Look for common parish container patterns
            selectors = [
                "[class*='parish']",
                "[class*='church']",
                "[class*='location']",
                "article",
                ".entry",
                "[id*='parish']"
            ]

            for selector in selectors:
                elements = soup.select(selector)
                if elements:
                    print(f"      📊 Found {len(elements)} elements with {selector}")

                    for element in elements[:20]:  # Limit to prevent timeout
                        parish_data = self._extract_parish_from_generic_element(element, url)
                        if parish_data:
                            parishes.append(parish_data)

                    if parishes:
                        break

        except Exception as e:
            print(f"    ⚠️ Generic extraction error: {str(e)[:100]}...")

        return parishes

    def _extract_parish_from_generic_element(self, element, base_url: str) -> Optional[ParishData]:
        """Extract parish data from generic element"""
        try:
            # Look for name in various heading tags
            name = None
            for tag in ['h1', 'h2', 'h3', 'h4', 'h5']:
                heading = element.find(tag)
                if heading:
                    name = self.clean_text(heading.get_text())
                    if name and len(name) > 2:
                        break

            if not name:
                # Try getting first significant text
                text_content = element.get_text()
                lines = [line.strip() for line in text_content.split('\n') if line.strip()]
                if lines:
                    name = lines[0]

            if not name or len(name) < 3:
                return None

            # Skip non-parish entries
            if not any(indicator in name.lower() for indicator in
                      ['parish', 'church', 'st.', 'saint', 'our lady', 'holy', 'cathedral']):
                return None

            # Extract other information
            element_text = element.get_text()

            phone = self.extract_phone(element_text)

            # Look for website
            website = None
            links = element.find_all('a', href=True)
            for link in links:
                href = link.get('href')
                if href.startswith('http') and not any(skip in href.lower()
                                                     for skip in ['facebook', 'twitter', 'instagram']):
                    website = href
                    break

            return ParishData(
                name=name,
                phone=phone,
                website=website,
                confidence_score=0.4,
                extraction_method="improved_generic_extraction"
            )

        except Exception as e:
            return None

print("✅ Base extractor classes loaded with enhanced Parish Finder implementation")

✅ Base extractor classes loaded with enhanced Parish Finder implementation


In [ ]:
# =============================================================================
# CELL 8.6: Debug and Fix Parish Finder Extraction (ADD THIS TO DEBUG THE ISSUE)
# =============================================================================

def debug_and_fix_parish_finder():
    """Debug the Parish Finder extraction and show what's happening"""

    print(f"🔍 DEBUGGING PARISH FINDER EXTRACTION ISSUE")
    print(f"{'='*60}")

    driver = setup_enhanced_driver()

    try:
        # Load the Parma parish finder page
        url = "https://www.parma.org/parishfinder"
        print(f"📥 Loading: {url}")
        driver.get(url)
        time.sleep(5)

        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')

        # Check the structure
        print(f"\n🔍 Analyzing page structure...")

        # Look for categories
        categories = soup.find('ul', id='categories')
        if categories:
            print(f"✅ Found #categories container")

            # Look for parish category
            parishes_li = categories.find('li', class_='category')
            if parishes_li:
                category_name_div = parishes_li.find('div', class_='categoryName')
                if category_name_div:
                    category_name = category_name_div.get_text().strip()
                    print(f"📂 Category name: '{category_name}'")

                # Look for sites within this category
                sites_ul = parishes_li.find('ul', class_='sites')
                if sites_ul:
                    sites = sites_ul.find_all('li', class_='site')
                    print(f"🏗️ Found {len(sites)} sites in parishes category")

                    # Analyze first few sites
                    for i, site in enumerate(sites[:5], 1):
                        print(f"\n  📍 Site {i}:")
                        print(f"      ID: {site.get('id', 'No ID')}")
                        print(f"      Data-latitude: {site.get('data-latitude', 'None')}")
                        print(f"      Data-longitude: {site.get('data-longitude', 'None')}")

                        # Look for name
                        name_div = site.find('div', class_='name')
                        if name_div:
                            name = name_div.get_text().strip()
                            print(f"      Name: '{name}'")
                        else:
                            print(f"      ❌ No name div found")

                        # Look for city
                        city_div = site.find('div', class_='city')
                        if city_div:
                            city = city_div.get_text().strip()
                            print(f"      City: '{city}'")
                        else:
                            print(f"      ❌ No city div found")

                        # Look for siteInfo
                        site_info = site.find('div', class_='siteInfo')
                        if site_info:
                            print(f"      ✅ Has siteInfo section")

                            # Check for detailed data within siteInfo
                            main_section = site_info.find('div', class_='main')
                            if main_section:
                                title_section = main_section.find('div', class_='title')
                                if title_section:
                                    # Check for address
                                    address_div = title_section.find('div', class_='address')
                                    if address_div:
                                        address = address_div.get_text().strip()
                                        print(f"        📍 Address: '{address}'")

                                    # Check for phone
                                    phone_holder = title_section.find('div', class_='phoneFaxHolder')
                                    if phone_holder:
                                        phone_span = phone_holder.find('span', class_='phone')
                                        if phone_span:
                                            phone = phone_span.get_text().strip()
                                            print(f"        📞 Phone: '{phone}'")
                        else:
                            print(f"      ❌ No siteInfo section")
                else:
                    print(f"❌ No ul.sites found in parishes category")
            else:
                print(f"❌ No li.category found in categories")
        else:
            print(f"❌ No #categories container found")

        # Now test the actual extraction
        print(f"\n🔧 Testing actual extraction logic...")

        # Create pattern detector
        detector = PatternDetector()
        pattern = detector.detect_pattern(html_content, url)
        print(f"📊 Detected pattern: {pattern.listing_type.value}")

        # Create extractor
        extractor = ParishFinderExtractor(pattern)

        # Test extraction
        parishes = extractor.extract(driver, soup, url)
        print(f"\n📊 Extraction results: {len(parishes)} parishes")

        # Show details of extracted parishes
        for i, parish in enumerate(parishes, 1):
            print(f"  {i}. {parish.name}")
            print(f"     City: {parish.city}")
            print(f"     Address: {parish.address}")
            print(f"     Phone: {parish.phone}")
            print(f"     Website: {parish.website}")
            print(f"     Coordinates: {parish.latitude}, {parish.longitude}")
            print(f"     Method: {parish.extraction_method}")
            print(f"     Confidence: {parish.confidence_score}")
            print()

        return parishes

    except Exception as e:
        print(f"❌ Debug error: {e}")
        return []

    finally:
        driver.quit()
        print("🧹 Debug WebDriver closed")

# Run the debug function
debug_results = debug_and_fix_parish_finder()

print("✅ Debug function completed")

🔍 DEBUGGING PARISH FINDER EXTRACTION ISSUE
🔧 Setting up Chrome driver...
✅ Chrome driver setup complete
📥 Loading: https://www.parma.org/parishfinder

🔍 Analyzing page structure...
✅ Found #categories container
📂 Category name: 'Parishes'
🏗️ Found 31 sites in parishes category

  📍 Site 1:
      ID: 20885
      Data-latitude: 41.6233383
      Data-longitude: -87.91294529999999
      Name: 'Annunciation of the Mother of God'
      City: 'Homer Glen'
      ✅ Has siteInfo section
        📍 Address: '14610 S. Will Cook Road, Homer Glen, Illinois, 60491-9212'
        📞 Phone: '708-645-0241'

  📍 Site 2:
      ID: 36949
      Data-latitude: 0.0
      Data-longitude: 0.0
      Name: 'Byzantine Catholic Outreach of Cincinnati'
      City: 'Cincinnati'
      ✅ Has siteInfo section
        📍 Address: 'St Anthony of Padua, Maronite Catholic Church,  Victory Pkwy, Cincinnati, OH 45206, Cincinnati, OH, 45206'

  📍 Site 3:
      ID: 20893
      Data-latitude: 41.439759
      Data-longitude: -91.0439

In [ ]:
# =============================================================================
# CELL 4.5: Improved Map Extractor (INSERT AFTER CELL 4)
# =============================================================================

class ImprovedInteractiveMapExtractor(BaseExtractor):
    """Improved extractor for JavaScript-powered maps with better error handling"""

    def extract(self, driver, soup: BeautifulSoup, url: str) -> List[ParishData]:
        parishes = []

        try:
            # Try to find map containers with more flexible selectors
            map_selectors = [
                "#map", ".map", ".parish-map", ".church-map",
                "[id*='map']", "[class*='map']",
                "#parish-finder", ".parish-finder"
            ]

            map_found = False
            for selector in map_selectors:
                try:
                    WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, selector))
                    )
                    map_found = True
                    print(f"    📍 Found map container: {selector}")
                    break
                except:
                    continue

            if not map_found:
                print(f"    ℹ️ No map container found, trying direct JS extraction...")

            # Method 1: Extract from JavaScript variables (works even without visible map)
            parishes.extend(self._extract_from_js_variables(driver))

            # Method 2: Look for parish data in script tags
            if not parishes:
                parishes.extend(self._extract_from_script_tags(soup))

            # Method 3: Extract from map markers (only if map found)
            if not parishes and map_found:
                parishes.extend(self._extract_from_markers(driver))

        except Exception as e:
            print(f"    ℹ️ Map extraction completed with info: {str(e)[:100]}...")

        return parishes

    def _extract_from_script_tags(self, soup: BeautifulSoup) -> List[ParishData]:
        """Extract parish data from script tags containing JSON"""
        parishes = []

        try:
            script_tags = soup.find_all('script')

            for script in script_tags:
                if not script.string:
                    continue

                script_content = script.string

                # Look for JSON-like data containing parish information
                if any(keyword in script_content.lower() for keyword in
                       ['parish', 'church', 'location', 'marker']):

                    # Try to extract JSON objects
                    import json

                    # Look for common patterns
                    patterns = [
                        r'parishes\s*[:=]\s*(\[.*?\])',
                        r'locations\s*[:=]\s*(\[.*?\])',
                        r'markers\s*[:=]\s*(\[.*?\])',
                        r'churches\s*[:=]\s*(\[.*?\])'
                    ]

                    for pattern in patterns:
                        matches = re.findall(pattern, script_content, re.DOTALL)
                        for match in matches:
                            try:
                                data = json.loads(match)
                                if isinstance(data, list):
                                    for item in data:
                                        parish = self._parse_js_parish_object(item)
                                        if parish:
                                            parishes.append(parish)
                            except:
                                continue

                        if parishes:
                            break

                if parishes:
                    break

        except Exception as e:
            print(f"    ℹ️ Script tag extraction info: {str(e)[:50]}...")

        return parishes

    def _extract_from_js_variables(self, driver) -> List[ParishData]:
        """Extract from common JavaScript variable names with better error handling"""
        parishes = []

        # Expanded list of common variable names
        js_vars = [
            "parishes", "parishData", "locations", "markers", "churchData",
            "parishList", "churches", "mapData", "data", "items",
            "parishInfo", "churchInfo", "mapMarkers", "points"
        ]

        for var_name in js_vars:
            try:
                js_data = driver.execute_script(f"""
                    try {{
                        return window.{var_name};
                    }} catch(e) {{
                        return null;
                    }}
                """)

                if js_data and isinstance(js_data, list) and len(js_data) > 0:
                    print(f"    📊 Found data in window.{var_name}: {len(js_data)} items")

                    for item in js_data:
                        parish = self._parse_js_parish_object(item)
                        if parish:
                            parishes.append(parish)

                    if parishes:
                        break

            except Exception as e:
                continue

        return parishes

    def _parse_js_parish_object(self, data: Dict) -> Optional[ParishData]:
        """Enhanced parsing of parish data from JavaScript object"""
        if not isinstance(data, dict):
            return None

        # Enhanced field mapping for name
        name = None
        for field in ['name', 'title', 'parishName', 'churchName', 'parish_name',
                      'church_name', 'label', 'text', 'Name', 'Title']:
            if field in data and data[field]:
                name = str(data[field]).strip()
                break

        if not name or len(name) < 3:
            return None

        # Skip non-parish entries
        if any(skip_word in name.lower() for skip_word in
               ['finder', 'directory', 'map', 'search', 'filter']):
            return None

        # Enhanced field mapping for other data
        address = None
        for field in ['address', 'location', 'fullAddress', 'street', 'addr']:
            if field in data and data[field]:
                address = str(data[field]).strip()
                break

        phone = None
        for field in ['phone', 'telephone', 'phoneNumber', 'tel', 'Phone']:
            if field in data and data[field]:
                phone = str(data[field]).strip()
                break

        website = None
        for field in ['website', 'url', 'link', 'web', 'Website', 'URL']:
            if field in data and data[field]:
                website = str(data[field]).strip()
                break

        # Coordinates
        lat = data.get('lat', data.get('latitude', data.get('Lat')))
        lng = data.get('lng', data.get('longitude', data.get('lon', data.get('Lng'))))

        return ParishData(
            name=name,
            address=address,
            phone=phone,
            website=website,
            latitude=float(lat) if lat else None,
            longitude=float(lng) if lng else None,
            confidence_score=0.8,
            extraction_method="improved_js_extraction"
        )

    def _extract_from_markers(self, driver) -> List[ParishData]:
        """Extract by clicking map markers with improved error handling"""
        parishes = []

        try:
            # More flexible marker selectors
            marker_selectors = [
                ".marker", ".leaflet-marker", ".map-marker",
                "[class*='marker']", ".gm-style-iw", ".mapboxgl-marker"
            ]

            markers = []
            for selector in marker_selectors:
                try:
                    found_markers = driver.find_elements(By.CSS_SELECTOR, selector)
                    if found_markers:
                        markers = found_markers
                        print(f"    📍 Found {len(markers)} markers using {selector}")
                        break
                except:
                    continue

            if not markers:
                print(f"    ℹ️ No clickable markers found")
                return parishes

            # Limit markers to avoid timeout
            for i, marker in enumerate(markers[:5]):  # Only try first 5
                try:
                    # Scroll marker into view
                    driver.execute_script("arguments[0].scrollIntoView(true);", marker)
                    time.sleep(0.5)

                    # Click marker
                    driver.execute_script("arguments[0].click();", marker)
                    time.sleep(1)

                    # Look for popup content with multiple selectors
                    popup_selectors = [
                        ".popup", ".info-window", ".mapboxgl-popup",
                        ".leaflet-popup", ".gm-style-iw-d"
                    ]

                    popup_text = None
                    for popup_selector in popup_selectors:
                        try:
                            popup = driver.find_element(By.CSS_SELECTOR, popup_selector)
                            popup_text = popup.text
                            break
                        except:
                            continue

                    if popup_text and len(popup_text) > 10:
                        parish_data = self._parse_popup_content(popup_text)
                        if parish_data:
                            parishes.append(parish_data)

                except Exception as e:
                    continue

        except Exception as e:
            print(f"    ℹ️ Marker extraction completed: {str(e)[:50]}...")

        return parishes

    def _parse_popup_content(self, popup_text: str) -> Optional[ParishData]:
        """Parse parish information from popup text"""
        lines = [line.strip() for line in popup_text.split('\n') if line.strip()]

        if not lines:
            return None

        name = lines[0]  # First line is usually the name

        # Skip if it doesn't look like a parish name
        if not any(indicator in name.lower() for indicator in
                  ['parish', 'church', 'st.', 'saint', 'our lady', 'holy', 'cathedral']):
            return None

        address = None
        phone = None

        # Look for address and phone in remaining lines
        for line in lines[1:]:
            if self.extract_phone(line):
                phone = self.extract_phone(line)
            elif re.search(r'\d+.*(?:street|st|avenue|ave|road|rd|drive|dr)', line, re.I):
                address = line

        return ParishData(
            name=name,
            address=address,
            phone=phone,
            confidence_score=0.6,
            extraction_method="marker_popup_extraction"
        )

# Replace the InteractiveMapExtractor in the main processing function
print("✅ Improved map extractor loaded with better error handling")

✅ Improved map extractor loaded with better error handling


In [ ]:
# =============================================================================
# CELL 5: Enhanced Database Integration Functions (UPDATED for Parish Finder data)
# =============================================================================

def prepare_parish_for_supabase(parish_data: ParishData, diocese_name: str, diocese_url: str, parish_directory_url: str) -> Dict:
    """Convert ParishData to format compatible with your existing Supabase schema (ENHANCED)"""

    # Use street address if available, otherwise fall back to full address or address
    street_address = parish_data.street_address or parish_data.full_address or parish_data.address

    return {
        'Name': parish_data.name,
        'Status': 'Parish',  # Default status
        'Deanery': None,  # Will be populated later if available
        'Street Address': street_address,
        'City': parish_data.city,
        'State': parish_data.state,
        'Zip Code': parish_data.zip_code,
        'Phone Number': parish_data.phone,
        'Web': parish_data.website,
        'diocese_url': diocese_url,
        'parish_directory_url': parish_directory_url,
        'parish_detail_url': parish_data.parish_detail_url,
        'extraction_method': parish_data.extraction_method,
        # Enhanced metadata fields
        'confidence_score': parish_data.confidence_score,
        'detail_extraction_success': parish_data.detail_extraction_success,
        'detail_extraction_error': parish_data.detail_extraction_error,
        'clergy_info': parish_data.clergy_info,
        'service_times': parish_data.service_times,
        'full_address': parish_data.full_address,
        'latitude': parish_data.latitude,
        'longitude': parish_data.longitude,
        'extracted_at': datetime.now().isoformat()
    }

def enhanced_safe_upsert_to_supabase(parishes: List[ParishData], diocese_name: str, diocese_url: str, parish_directory_url: str):
    """Enhanced version of your existing Supabase upsert function with Parish Finder support"""

    if not supabase:
        print("  ❌ Supabase not available")
        return False

    success_count = 0
    detail_success_count = 0
    skipped_count = 0

    for parish in parishes:
        try:
            # Enhanced filtering for non-parish items
            skip_terms = [
                'finder', 'contact', 'chancery', 'pastoral center', 'tv mass',
                'directory', 'search', 'filter', 'map', 'diocese', 'bishop',
                'office', 'center', 'no parish registration', 'archdiocese'
            ]

            if any(skip_word in parish.name.lower() for skip_word in skip_terms):
                print(f"    ⏭️ Skipped: {parish.name} (not a parish)")
                skipped_count += 1
                continue

            # Must have a meaningful name to proceed
            if not parish.name or len(parish.name.strip()) < 3:
                print(f"    ⏭️ Skipped: Invalid name for parish")
                skipped_count += 1
                continue

            # Convert to your existing schema format
            supabase_data = prepare_parish_for_supabase(parish, diocese_name, diocese_url, parish_directory_url)

            # Remove None values and empty strings, but keep boolean False and numeric 0
            clean_data = {}
            for k, v in supabase_data.items():
                if v is not None and v != "":
                    clean_data[k] = v
                elif isinstance(v, (bool, int, float)):
                    clean_data[k] = v

            # Must have a name to proceed
            if not clean_data.get('Name') or len(clean_data.get('Name', '')) < 3:
                print(f"    ⏭️ Skipped: Invalid name after cleaning")
                skipped_count += 1
                continue

            # Use your existing upsert logic
            response = supabase.table('Parishes').insert(clean_data).execute()

            if hasattr(response, 'error') and response.error:
                print(f"    ❌ Database error for {parish.name}: {response.error}")
            else:
                success_count += 1
                if parish.detail_extraction_success:
                    detail_success_count += 1
                    detail_indicator = "📍"  # Pin icon for detailed info
                else:
                    detail_indicator = "📌"  # Basic pin for basic info

                # Show confidence and extraction method
                method_short = parish.extraction_method.replace('_extraction', '').replace('_', ' ')
                print(f"    ✅ {detail_indicator} Saved: {parish.name}")
                print(f"        📊 Method: {method_short}, Confidence: {parish.confidence_score:.2f}")

                # Show what fields were captured for Parish Finder data
                captured_fields = []
                if parish.city: captured_fields.append("city")
                if parish.address or parish.street_address or parish.full_address: captured_fields.append("address")
                if parish.phone: captured_fields.append("phone")
                if parish.website: captured_fields.append("website")
                if parish.latitude and parish.longitude: captured_fields.append("coordinates")
                if parish.clergy_info: captured_fields.append("clergy")
                if parish.service_times: captured_fields.append("schedule")

                if captured_fields:
                    print(f"        📋 Fields: {', '.join(captured_fields)}")

        except Exception as e:
            print(f"    ❌ Error saving {parish.name}: {e}")

    print(f"  📊 Results: {success_count} saved, {skipped_count} skipped, {detail_success_count} with detailed info")
    if success_count > 0:
        success_rate = (success_count / (success_count + skipped_count)) * 100
        print(f"  📈 Success rate: {success_rate:.1f}%")

    return success_count > 0

def analyze_parish_finder_quality(parishes: List[ParishData]) -> Dict:
    """Analyze the quality of Parish Finder extraction specifically"""

    if not parishes:
        return {'error': 'No parishes to analyze'}

    total_parishes = len(parishes)

    # Parish Finder specific analysis
    analysis = {
        'total_parishes': total_parishes,
        'extraction_methods': {},
        'parish_finder_specific': {
            'has_coordinates': sum(1 for p in parishes if p.latitude and p.longitude),
            'has_city': sum(1 for p in parishes if p.city),
            'has_site_info': sum(1 for p in parishes if p.detail_extraction_success),
            'confidence_distribution': {
                'high_confidence': sum(1 for p in parishes if p.confidence_score >= 0.8),
                'medium_confidence': sum(1 for p in parishes if 0.5 <= p.confidence_score < 0.8),
                'low_confidence': sum(1 for p in parishes if p.confidence_score < 0.5)
            }
        },
        'data_completeness': {
            'names_present': sum(1 for p in parishes if p.name and len(p.name) > 2),
            'cities_present': sum(1 for p in parishes if p.city),
            'addresses_present': sum(1 for p in parishes if p.street_address or p.full_address or p.address),
            'phones_present': sum(1 for p in parishes if p.phone),
            'websites_present': sum(1 for p in parishes if p.website),
            'coordinates_present': sum(1 for p in parishes if p.latitude and p.longitude),
            'clergy_info_present': sum(1 for p in parishes if p.clergy_info)
        }
    }

    # Track extraction methods used
    for parish in parishes:
        method = parish.extraction_method
        analysis['extraction_methods'][method] = analysis['extraction_methods'].get(method, 0) + 1

    # Calculate percentages
    analysis['data_completeness_percentages'] = {
        f"{key}_percentage": (value / total_parishes * 100)
        for key, value in analysis['data_completeness'].items()
    }

    # Overall quality score specific to Parish Finder
    basic_score = (analysis['data_completeness']['names_present'] +
                  analysis['data_completeness']['cities_present']) / (total_parishes * 2) * 100

    enhanced_score = (analysis['data_completeness']['addresses_present'] +
                     analysis['data_completeness']['phones_present'] +
                     analysis['data_completeness']['coordinates_present']) / (total_parishes * 3) * 100

    analysis['parish_finder_quality_score'] = (basic_score + enhanced_score) / 2

    return analysis

def export_parish_finder_debug_data(parishes: List[ParishData], diocese_name: str) -> str:
    """Export Parish Finder data with debug information"""

    import csv
    import os

    if not parishes:
        return None

    # Create filename
    safe_diocese_name = "".join(c for c in diocese_name if c.isalnum() or c in (' ', '-', '_')).rstrip()
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"parish_finder_debug_{safe_diocese_name}_{timestamp}.csv"

    try:
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = [
                'Name', 'City', 'Address', 'Phone', 'Website',
                'Latitude', 'Longitude', 'Clergy Info', 'Confidence Score',
                'Extraction Method', 'Detail Extraction Success',
                'Has Coordinates', 'Data Quality Score'
            ]

            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

            for parish in parishes:
                # Calculate a simple data quality score
                quality_fields = [
                    bool(parish.name and len(parish.name) > 2),
                    bool(parish.city),
                    bool(parish.address or parish.street_address or parish.full_address),
                    bool(parish.phone),
                    bool(parish.website),
                    bool(parish.latitude and parish.longitude)
                ]
                quality_score = sum(quality_fields) / len(quality_fields) * 100

                writer.writerow({
                    'Name': parish.name,
                    'City': parish.city,
                    'Address': parish.address or parish.street_address or parish.full_address,
                    'Phone': parish.phone,
                    'Website': parish.website,
                    'Latitude': parish.latitude,
                    'Longitude': parish.longitude,
                    'Clergy Info': parish.clergy_info,
                    'Confidence Score': parish.confidence_score,
                    'Extraction Method': parish.extraction_method,
                    'Detail Extraction Success': parish.detail_extraction_success,
                    'Has Coordinates': bool(parish.latitude and parish.longitude),
                    'Data Quality Score': f"{quality_score:.1f}%"
                })

        print(f"📁 Parish Finder debug data exported to: {filename}")
        return filename

    except Exception as e:
        print(f"❌ Error exporting debug data: {e}")
        return None

print("✅ Enhanced database integration functions loaded with Parish Finder support")

✅ Enhanced database integration functions loaded with Parish Finder support


In [ ]:
# =============================================================================
# CELL 5.5: WebDriver Setup Function (ADD THIS BEFORE CELL 6)
# =============================================================================

def setup_enhanced_driver():
    """Set up Chrome WebDriver with options optimized for parish extraction"""

    print("🔧 Setting up enhanced Chrome WebDriver...")

    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--disable-plugins')
    chrome_options.add_argument('--disable-images')  # Speed up loading
    chrome_options.add_argument('--disable-javascript-harmony-shipping')
    chrome_options.add_argument('--disable-background-timer-throttling')
    chrome_options.add_argument('--disable-backgrounding-occluded-windows')
    chrome_options.add_argument('--disable-renderer-backgrounding')

    # User agent to avoid blocking
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.set_page_load_timeout(30)
        driver.implicitly_wait(5)

        print("✅ Chrome WebDriver initialized successfully")
        return driver

    except Exception as e:
        print(f"❌ Failed to initialize WebDriver: {e}")
        raise

print("✅ WebDriver setup function loaded")

✅ WebDriver setup function loaded


In [ ]:
# =============================================================================
# CELL 6: Enhanced Master Processing Function with Parish Finder Priority (UPDATED)
# =============================================================================

def process_diocese_with_detailed_extraction(diocese_info: Dict, driver) -> Dict:
    """
    Enhanced processing function that extracts detailed parish information
    by navigating to individual parish detail pages (UPDATED for Parish Finder)
    """

    diocese_url = diocese_info['url']
    diocese_name = diocese_info['name']
    parish_directory_url = diocese_info['parish_directory_url']

    print(f"\n{'='*60}")
    print(f"🔍 ENHANCED DETAILED PROCESSING: {diocese_name}")
    print(f"📍 Main URL: {diocese_url}")
    print(f"📂 Parish Directory URL: {parish_directory_url}")
    print(f"{'='*60}")

    result = {
        'diocese_name': diocese_name,
        'diocese_url': diocese_url,
        'parish_directory_url': parish_directory_url,
        'timestamp': datetime.now().isoformat(),
        'pattern_detected': None,
        'parishes_found': [],
        'success': False,
        'extraction_methods_used': [],
        'processing_time': 0,
        'errors': [],
        # Enhanced tracking
        'detail_extraction_stats': {
            'attempted': 0,
            'successful': 0,
            'failed': 0,
            'success_rate': 0.0
        },
        'field_extraction_stats': {
            'addresses_extracted': 0,
            'phones_extracted': 0,
            'websites_extracted': 0,
            'zip_codes_extracted': 0,
            'clergy_info_extracted': 0,
            'service_times_extracted': 0
        }
    }

    start_time = time.time()

    try:
        # Step 1: Load the parish directory page
        print("  📥 Loading parish directory page...")
        driver.get(parish_directory_url)
        time.sleep(3)  # Give time for JS to load

        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')

        # Step 2: Detect pattern
        print("  🔍 Detecting website pattern...")
        detector = PatternDetector()
        pattern = detector.detect_pattern(html_content, parish_directory_url)

        result['pattern_detected'] = {
            'platform': pattern.platform.value,
            'listing_type': pattern.listing_type.value,
            'confidence': pattern.confidence_score,
            'extraction_method': pattern.extraction_method,
            'javascript_required': pattern.javascript_required,
            'notes': pattern.notes
        }

        print(f"    📋 Platform: {pattern.platform.value}")
        print(f"    📊 Listing Type: {pattern.listing_type.value}")
        print(f"    🎯 Confidence: {pattern.confidence_score:.2f}")
        print(f"    ⚙️ Method: {pattern.extraction_method}")

        # Step 3: Extract parishes with detailed information
        parishes = []

        # Try extractors in order of specificity (UPDATED to prioritize Parish Finder)
        extractors_to_try = []

        # Primary extractor based on detected pattern (enhanced versions)
        if pattern.listing_type == ParishListingType.PARISH_FINDER:
            extractors_to_try.append(('ParishFinderExtractor', ParishFinderExtractor(pattern)))
        elif pattern.listing_type == ParishListingType.DIOCESE_CARD_LAYOUT:
            extractors_to_try.append(('EnhancedDiocesesCardExtractor', EnhancedDiocesesCardExtractor(pattern)))
        elif pattern.listing_type == ParishListingType.STATIC_TABLE:
            extractors_to_try.append(('TableExtractor', TableExtractor(pattern)))
        elif pattern.listing_type == ParishListingType.INTERACTIVE_MAP:
            extractors_to_try.append(('ImprovedInteractiveMapExtractor', ImprovedInteractiveMapExtractor(pattern)))

        # Add other extractors as fallbacks in priority order
        fallback_extractors = [
            ('ParishFinderExtractor', ParishFinderExtractor(pattern)),
            ('EnhancedDiocesesCardExtractor', EnhancedDiocesesCardExtractor(pattern)),
            ('TableExtractor', TableExtractor(pattern)),
            ('ImprovedInteractiveMapExtractor', ImprovedInteractiveMapExtractor(pattern)),
            ('ImprovedGenericExtractor', ImprovedGenericExtractor(pattern))
        ]

        # Add fallbacks that aren't already in the list
        for name, extractor in fallback_extractors:
            if not any(existing_name == name for existing_name, _ in extractors_to_try):
                extractors_to_try.append((name, extractor))

        # Try each extractor until we find parishes
        for extractor_name, extractor in extractors_to_try:
            try:
                print(f"  🔄 Trying {extractor_name}...")
                current_parishes = extractor.extract(driver, soup, parish_directory_url)

                if current_parishes:
                    parishes.extend(current_parishes)
                    result['extraction_methods_used'].append(extractor_name)
                    print(f"    ✅ {extractor_name} found {len(current_parishes)} parishes")

                    # If we found parishes with a specialized extractor, prefer those results
                    if extractor_name in ['ParishFinderExtractor', 'EnhancedDiocesesCardExtractor'] and len(current_parishes) > 3:
                        print(f"    🎯 Using {extractor_name} - stopping extraction")
                        break

                    # If we found a good number of parishes with any method, stop
                    if len(parishes) > 10:
                        break
                else:
                    print(f"    ⚠️ {extractor_name} found no parishes")

            except Exception as e:
                print(f"    ❌ {extractor_name} failed: {str(e)[:100]}")
                result['errors'].append(f"{extractor_name}: {str(e)[:100]}")

        # Step 4: Process results and calculate statistics
        if parishes:
            # Remove duplicates and validate
            unique_parishes = []
            seen_names = set()

            for parish in parishes:
                name_key = parish.name.lower().strip()
                if name_key not in seen_names and len(parish.name) > 2:
                    # Set the source URLs for each parish
                    parish.diocese_url = diocese_url
                    parish.parish_directory_url = parish_directory_url
                    unique_parishes.append(parish)
                    seen_names.add(name_key)

            result['parishes_found'] = unique_parishes
            result['success'] = True

            # Calculate detailed extraction statistics
            total_parishes = len(unique_parishes)
            detailed_successful = sum(1 for p in unique_parishes if p.detail_extraction_success)
            detailed_failed = sum(1 for p in unique_parishes if hasattr(p, 'detail_extraction_success') and not p.detail_extraction_success)

            result['detail_extraction_stats'] = {
                'attempted': total_parishes,
                'successful': detailed_successful,
                'failed': detailed_failed,
                'success_rate': (detailed_successful / total_parishes * 100) if total_parishes > 0 else 0
            }

            # Calculate field extraction statistics
            result['field_extraction_stats'] = {
                'addresses_extracted': sum(1 for p in unique_parishes if p.street_address or p.full_address or p.address),
                'phones_extracted': sum(1 for p in unique_parishes if p.phone),
                'websites_extracted': sum(1 for p in unique_parishes if p.website),
                'zip_codes_extracted': sum(1 for p in unique_parishes if p.zip_code),
                'clergy_info_extracted': sum(1 for p in unique_parishes if p.clergy_info),
                'service_times_extracted': sum(1 for p in unique_parishes if p.service_times)
            }

            print(f"  ✅ Found {len(unique_parishes)} unique parishes")
            print(f"  📊 Detail extraction: {detailed_successful}/{total_parishes} successful ({result['detail_extraction_stats']['success_rate']:.1f}%)")

            # Show field extraction summary
            field_stats = result['field_extraction_stats']
            print(f"  📋 Field extraction summary:")
            print(f"      📍 Addresses: {field_stats['addresses_extracted']}/{total_parishes}")
            print(f"      📞 Phones: {field_stats['phones_extracted']}/{total_parishes}")
            print(f"      🌐 Websites: {field_stats['websites_extracted']}/{total_parishes}")
            print(f"      📮 Zip Codes: {field_stats['zip_codes_extracted']}/{total_parishes}")
            print(f"      👥 Clergy Info: {field_stats['clergy_info_extracted']}/{total_parishes}")
            print(f"      ⏰ Service Times: {field_stats['service_times_extracted']}/{total_parishes}")

            # Step 5: Save to database
            if unique_parishes:
                print("  💾 Saving enhanced parish data to database...")
                enhanced_safe_upsert_to_supabase(unique_parishes, diocese_name, diocese_url, parish_directory_url)

        else:
            print("  ❌ No parishes found with any extraction method")
            result['success'] = False

    except Exception as e:
        error_msg = str(e)
        result['errors'].append(error_msg)
        print(f"  ❌ Processing error: {error_msg}")

    finally:
        result['processing_time'] = time.time() - start_time
        print(f"  ⏱️ Completed in {result['processing_time']:.1f}s")

    return result

print("✅ Enhanced master processing function loaded with Parish Finder priority")

✅ Enhanced master processing function loaded with Parish Finder priority


In [ ]:
# =============================================================================
# CELL 7: Main Execution Using Existing Parish Directory URLs (MODIFIED TO EXCLUDE PROCESSED DIOCESES)
# =============================================================================

# Get dioceses WITH their parish directory URLs from your existing data
if supabase:
    try:
        print("📥 Fetching dioceses with parish directory URLs from database...")

        # Join dioceses with their parish directory URLs
        response = supabase.table('DiocesesParishDirectory').select(
            'diocese_url, parish_directory_url'
        ).not_.is_('parish_directory_url', 'null').not_.eq('parish_directory_url', '').execute()

        diocese_directory_data = response.data if response.data else []
        print(f"📊 Found {len(diocese_directory_data)} dioceses with parish directory URLs")

        # Get diocese names from the main table
        if diocese_directory_data:
            diocese_urls = [item['diocese_url'] for item in diocese_directory_data]

            # Get diocese names for these URLs
            diocese_names_response = supabase.table('Dioceses').select(
                'Website, Name'
            ).in_('Website', diocese_urls).execute()

            diocese_names_data = diocese_names_response.data if diocese_names_response.data else []

            # Create a mapping of URL to name
            url_to_name = {item['Website']: item['Name'] for item in diocese_names_data}

            # NEW: Check which dioceses already have 5+ parishes in the database
            print("🔍 Checking for dioceses that already have 5+ parishes extracted...")

            # Get all parishes with their parish_directory_url
            parish_counts_response = supabase.table('Parishes').select(
                'parish_directory_url'
            ).not_.is_('parish_directory_url', 'null').not_.eq('parish_directory_url', '').execute()

            parish_counts_data = parish_counts_response.data if parish_counts_response.data else []

            # Count parishes by directory URL
            directory_url_counts = {}
            for parish in parish_counts_data:
                directory_url = parish.get('parish_directory_url')
                if directory_url:
                    directory_url_counts[directory_url] = directory_url_counts.get(directory_url, 0) + 1

            # Combine the data and filter out already processed dioceses
            dioceses_to_process = []
            dioceses_skipped = []
            first_kept_shown = False

            for item in diocese_directory_data:
                diocese_url = item['diocese_url']
                parish_directory_url = item['parish_directory_url']
                diocese_name = url_to_name.get(diocese_url, 'Unknown Diocese')

                # Check if this diocese already has 5+ parishes
                existing_parish_count = directory_url_counts.get(parish_directory_url, 0)

                if existing_parish_count >= 5:
                    dioceses_skipped.append({
                        'name': diocese_name,
                        'url': diocese_url,
                        'parish_directory_url': parish_directory_url,
                        'existing_parish_count': existing_parish_count
                    })
                    print(f"  ⏭️ SKIPPED {diocese_name}: {existing_parish_count} parishes already extracted")
                else:
                    dioceses_to_process.append({
                        'name': diocese_name,
                        'url': diocese_url,
                        'parish_directory_url': parish_directory_url,
                        'existing_parish_count': existing_parish_count
                    })
                    if not first_kept_shown:
                        print(f"  ✅ FIRST TO PROCESS: {diocese_name} ({existing_parish_count} existing parishes)")
                        first_kept_shown = True

            print(f"\n📊 FILTERING RESULTS:")
            print(f"  - Dioceses to process: {len(dioceses_to_process)}")
            print(f"  - Dioceses skipped (5+ parishes): {len(dioceses_skipped)}")

            # MODIFIED: Take the first N dioceses instead of random sampling
            if len(dioceses_to_process) > MAX_DIOCESES_TO_PROCESS:
                print(f"📊 Taking first {MAX_DIOCESES_TO_PROCESS} dioceses from {len(dioceses_to_process)} available")
                dioceses_to_process = dioceses_to_process[:MAX_DIOCESES_TO_PROCESS]
            else:
                print(f"📊 Processing all {len(dioceses_to_process)} available dioceses")

            print(f"📊 Selected {len(dioceses_to_process)} dioceses for enhanced processing")

            # Display what we're about to process
            if dioceses_to_process:
                print(f"\n📋 Dioceses to process:")
                for i, diocese in enumerate(dioceses_to_process, 1):
                    existing_count = diocese.get('existing_parish_count', 0)
                    existing_info = f" ({existing_count} existing)" if existing_count > 0 else " (new)"
                    print(f"  {i}. {diocese['name']}{existing_info}")
                    print(f"     Main URL: {diocese['url']}")
                    print(f"     Parish Directory: {diocese['parish_directory_url']}")

            # Display summary of skipped dioceses
            if dioceses_skipped:
                print(f"\n📋 Dioceses skipped (already have 5+ parishes):")
                for diocese in dioceses_skipped[:5]:  # Show first 5
                    print(f"  - {diocese['name']}: {diocese['existing_parish_count']} parishes")
                if len(dioceses_skipped) > 5:
                    print(f"  ... and {len(dioceses_skipped) - 5} more")

        else:
            dioceses_to_process = []

    except Exception as e:
        print(f"❌ Error fetching dioceses with parish directories: {e}")
        dioceses_to_process = []
else:
    print("❌ No Supabase connection, using test data")
    dioceses_to_process = []

print("✅ Enhanced processing function loaded (MODIFIED to exclude dioceses with 5+ existing parishes)")

📥 Fetching dioceses with parish directory URLs from database...
📊 Found 192 dioceses with parish directory URLs
🔍 Checking for dioceses that already have 5+ parishes extracted...
  ⏭️ SKIPPED Diocese of Salt Lake City: 79 parishes already extracted
  ✅ FIRST TO PROCESS: Eparchy of Parma (0 existing parishes)

📊 FILTERING RESULTS:
  - Dioceses to process: 191
  - Dioceses skipped (5+ parishes): 1
📊 Taking first 1 dioceses from 191 available
📊 Selected 1 dioceses for enhanced processing

📋 Dioceses to process:
  1. Eparchy of Parma (new)
     Main URL: http://www.parma.org/
     Parish Directory: https://www.parma.org/parishfinder

📋 Dioceses skipped (already have 5+ parishes):
  - Diocese of Salt Lake City: 79 parishes
✅ Enhanced processing function loaded (MODIFIED to exclude dioceses with 5+ existing parishes)


In [ ]:
# =============================================================================
# CELL 8: Execute Enhanced Processing with Detail Extraction (UPDATED)
# =============================================================================

if dioceses_to_process:
    print(f"\n🚀 Starting ENHANCED pattern-based processing with DETAILED parish extraction...")
    print(f"📋 This will click on each parish card to extract complete information:")
    print(f"    📍 Street addresses and zip codes")
    print(f"    📞 Phone numbers")
    print(f"    🌐 Parish websites")
    print(f"    👥 Clergy information")
    print(f"    ⏰ Service times and schedules")

    # Initialize driver
    driver = setup_enhanced_driver()

    # Track enhanced results
    all_results = []
    summary_stats = {
        'total_dioceses': len(dioceses_to_process),
        'successful_extractions': 0,
        'total_parishes_found': 0,
        'pattern_distribution': {},
        'extraction_method_usage': {},
        'average_confidence': 0.0,
        # Enhanced statistics
        'total_detail_extractions_attempted': 0,
        'total_detail_extractions_successful': 0,
        'overall_detail_success_rate': 0.0,
        'field_extraction_totals': {
            'addresses_extracted': 0,
            'phones_extracted': 0,
            'websites_extracted': 0,
            'zip_codes_extracted': 0,
            'clergy_info_extracted': 0,
            'service_times_extracted': 0
        }
    }

    try:
        for i, diocese_info in enumerate(dioceses_to_process, 1):
            print(f"\n📍 Diocese {i}/{len(dioceses_to_process)}")

            # Process with enhanced detailed extraction system
            result = process_diocese_with_detailed_extraction(diocese_info, driver)
            all_results.append(result)

            # Update summary statistics
            if result['success']:
                summary_stats['successful_extractions'] += 1
                summary_stats['total_parishes_found'] += len(result['parishes_found'])

                # Enhanced statistics tracking
                detail_stats = result.get('detail_extraction_stats', {})
                summary_stats['total_detail_extractions_attempted'] += detail_stats.get('attempted', 0)
                summary_stats['total_detail_extractions_successful'] += detail_stats.get('successful', 0)

                # Field extraction statistics
                field_stats = result.get('field_extraction_stats', {})
                for field, count in field_stats.items():
                    if field in summary_stats['field_extraction_totals']:
                        summary_stats['field_extraction_totals'][field] += count

                # Track pattern distribution
                if result['pattern_detected']:
                    pattern_key = f"{result['pattern_detected']['platform']}_{result['pattern_detected']['listing_type']}"
                    summary_stats['pattern_distribution'][pattern_key] = summary_stats['pattern_distribution'].get(pattern_key, 0) + 1

                # Track extraction methods
                for method in result['extraction_methods_used']:
                    summary_stats['extraction_method_usage'][method] = summary_stats['extraction_method_usage'].get(method, 0) + 1

            # Be respectful - pause between requests (longer pause due to detail extraction)
            if i < len(dioceses_to_process):
                time.sleep(3)

    finally:
        # Clean up
        driver.quit()
        print("\n🧹 WebDriver closed")

    # Calculate final enhanced statistics
    if summary_stats['successful_extractions'] > 0:
        summary_stats['success_rate'] = (summary_stats['successful_extractions'] / summary_stats['total_dioceses']) * 100
        summary_stats['avg_parishes_per_diocese'] = summary_stats['total_parishes_found'] / summary_stats['successful_extractions']

    if summary_stats['total_detail_extractions_attempted'] > 0:
        summary_stats['overall_detail_success_rate'] = (summary_stats['total_detail_extractions_successful'] / summary_stats['total_detail_extractions_attempted']) * 100

    # =============================================================================
    # ENHANCED RESULTS DISPLAY
    # =============================================================================

    print(f"\n{'='*70}")
    print(f"📊 ENHANCED DETAILED EXTRACTION SUMMARY")
    print(f"{'='*70}")
    print(f"Total dioceses processed: {summary_stats['total_dioceses']}")
    print(f"Successful extractions: {summary_stats['successful_extractions']}")
    print(f"Success rate: {summary_stats.get('success_rate', 0):.1f}%")
    print(f"Total parishes found: {summary_stats['total_parishes_found']}")
    if summary_stats['successful_extractions'] > 0:
        print(f"Average parishes per diocese: {summary_stats.get('avg_parishes_per_diocese', 0):.1f}")

    print(f"\n📍 DETAILED EXTRACTION PERFORMANCE:")
    print(f"Detail extractions attempted: {summary_stats['total_detail_extractions_attempted']}")
    print(f"Detail extractions successful: {summary_stats['total_detail_extractions_successful']}")
    print(f"Overall detail success rate: {summary_stats['overall_detail_success_rate']:.1f}%")

    print(f"\n📋 FIELD EXTRACTION TOTALS:")
    field_totals = summary_stats['field_extraction_totals']
    total_parishes = summary_stats['total_parishes_found']
    if total_parishes > 0:
        print(f"  📍 Street Addresses: {field_totals['addresses_extracted']}/{total_parishes} ({field_totals['addresses_extracted']/total_parishes*100:.1f}%)")
        print(f"  📞 Phone Numbers: {field_totals['phones_extracted']}/{total_parishes} ({field_totals['phones_extracted']/total_parishes*100:.1f}%)")
        print(f"  🌐 Websites: {field_totals['websites_extracted']}/{total_parishes} ({field_totals['websites_extracted']/total_parishes*100:.1f}%)")
        print(f"  📮 Zip Codes: {field_totals['zip_codes_extracted']}/{total_parishes} ({field_totals['zip_codes_extracted']/total_parishes*100:.1f}%)")
        print(f"  👥 Clergy Info: {field_totals['clergy_info_extracted']}/{total_parishes} ({field_totals['clergy_info_extracted']/total_parishes*100:.1f}%)")
        print(f"  ⏰ Service Times: {field_totals['service_times_extracted']}/{total_parishes} ({field_totals['service_times_extracted']/total_parishes*100:.1f}%)")

    print(f"\n📈 Pattern Distribution:")
    for pattern, count in summary_stats['pattern_distribution'].items():
        percentage = (count / summary_stats['total_dioceses']) * 100
        print(f"  {pattern.replace('_', ' ').title()}: {count} dioceses ({percentage:.1f}%)")

    print(f"\n🔧 Extraction Method Usage:")
    for method, count in summary_stats['extraction_method_usage'].items():
        enhanced_indicator = "🔥" if "Enhanced" in method else ""
        print(f"  {enhanced_indicator} {method}: {count} times")

    print(f"\n🔍 Detailed Results:")
    for result in all_results:
        status = "✅" if result['success'] else "❌"
        parish_count = len(result['parishes_found'])

        # Detail extraction summary
        detail_stats = result.get('detail_extraction_stats', {})
        detail_success = detail_stats.get('successful', 0)
        detail_total = detail_stats.get('attempted', 0)
        detail_rate = detail_stats.get('success_rate', 0)

        pattern_info = ""
        if result['pattern_detected']:
            pattern_info = f" [{result['pattern_detected']['platform']} / {result['pattern_detected']['listing_type']}]"

        print(f"  {status} {result['diocese_name']}: {parish_count} parishes{pattern_info}")
        print(f"      📊 Detail extraction: {detail_success}/{detail_total} successful ({detail_rate:.1f}%)")
        print(f"      🌐 Main URL: {result['diocese_url']}")
        print(f"      📂 Parish Directory: {result['parish_directory_url']}")

        if result['extraction_methods_used']:
            methods = ', '.join(result['extraction_methods_used'])
            print(f"      🔧 Methods: {methods}")

        # Show field extraction summary for this diocese
        field_stats = result.get('field_extraction_stats', {})
        if any(field_stats.values()):
            field_summary = []
            if field_stats.get('addresses_extracted', 0) > 0:
                field_summary.append(f"📍{field_stats['addresses_extracted']} addresses")
            if field_stats.get('phones_extracted', 0) > 0:
                field_summary.append(f"📞{field_stats['phones_extracted']} phones")
            if field_stats.get('websites_extracted', 0) > 0:
                field_summary.append(f"🌐{field_stats['websites_extracted']} websites")
            if field_stats.get('zip_codes_extracted', 0) > 0:
                field_summary.append(f"📮{field_stats['zip_codes_extracted']} zips")

            if field_summary:
                print(f"      📋 Fields: {', '.join(field_summary[:4])}")

        if result['errors']:
            for error in result['errors']:
                print(f"      ❌ Error: {error[:100]}...")
        print()  # Add blank line between dioceses

    # Save enhanced summary to file
    summary_filename = f"enhanced_extraction_summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(summary_filename, 'w') as f:
        json.dump({
            'summary_stats': summary_stats,
            'detailed_results': all_results
        }, f, indent=2, default=str)
    print(f"💾 Enhanced detailed results saved to: {summary_filename}")

else:
    print("❌ No dioceses with parish directory URLs found to process")

print(f"\n🎉 Enhanced pattern-based extraction with DETAILED parish information complete!")
print(f"📊 Successfully extracted detailed information for parishes including addresses, phones, websites, and more!")
print(f"{'='*70}")


🚀 Starting ENHANCED pattern-based processing with DETAILED parish extraction...
📋 This will click on each parish card to extract complete information:
    📍 Street addresses and zip codes
    📞 Phone numbers
    🌐 Parish websites
    👥 Clergy information
    ⏰ Service times and schedules
🔧 Setting up enhanced Chrome WebDriver...
✅ Chrome WebDriver initialized successfully

📍 Diocese 1/1

🔍 ENHANCED DETAILED PROCESSING: Eparchy of Parma
📍 Main URL: http://www.parma.org/
📂 Parish Directory URL: https://www.parma.org/parishfinder
  📥 Loading parish directory page...
  🔍 Detecting website pattern...
    📋 Platform: ecatholic
    📊 Listing Type: parish_finder
    🎯 Confidence: 0.95
    ⚙️ Method: parish_finder_extraction
  🔄 Trying ParishFinderExtractor...
    📍 Parish finder interface detected
    📊 Found 125 parish elements using li.site
      ⚠️ Skipped element 1: No valid parish data
      ⚠️ Skipped element 2: No valid parish data
      ⚠️ Skipped element 3: No valid parish data
     

In [ ]:
# =============================================================================
# CELL 8.5: Debug Parish Finder Elements (ADD AFTER EXECUTION TO TROUBLESHOOT)
# =============================================================================

def debug_parish_finder_structure(url: str = "https://www.parma.org/parishfinder"):
    """Debug function to examine the Parish Finder structure"""

    print(f"🔍 DEBUGGING PARISH FINDER STRUCTURE")
    print(f"📍 URL: {url}")
    print(f"{'='*60}")

    driver = setup_enhanced_driver()

    try:
        # Load the page
        print("📥 Loading page...")
        driver.get(url)
        time.sleep(5)  # Give extra time for JS to load

        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')

        # Check for parish elements
        print("\n🔍 Looking for parish elements...")

        # Check different selectors
        selectors_to_check = [
            ("li.site", soup.find_all('li', class_='site')),
            (".site", soup.find_all(class_='site')),
            ("li[id]", soup.find_all('li', id=True)),
            ("#categories li", soup.select('#categories li')),
            ("ul.sites li", soup.select('ul.sites li')),
            ("[data-latitude]", soup.find_all(attrs={"data-latitude": True}))
        ]

        for selector, elements in selectors_to_check:
            print(f"  📊 {selector}: {len(elements)} elements")

            if elements and len(elements) > 0:
                print(f"    🔍 First element structure:")
                first_element = elements[0]

                # Show the element's HTML structure (first 500 chars)
                element_html = str(first_element)[:500]
                print(f"      HTML: {element_html}...")

                # Look for name
                name_elem = first_element.find('div', class_='name')
                if name_elem:
                    print(f"      👤 Name found: {name_elem.get_text().strip()}")
                else:
                    print(f"      ❌ No .name div found")

                # Look for city
                city_elem = first_element.find('div', class_='city')
                if city_elem:
                    print(f"      🏙️ City found: {city_elem.get_text().strip()}")
                else:
                    print(f"      ❌ No .city div found")

                # Look for siteInfo
                site_info = first_element.find('div', class_='siteInfo')
                if site_info:
                    print(f"      📋 siteInfo found: {len(str(site_info))} chars")

                    # Look inside siteInfo for detailed data
                    title_section = site_info.find('div', class_='title')
                    if title_section:
                        print(f"        📍 title section found")

                        address_div = title_section.find('div', class_='address')
                        if address_div:
                            print(f"        🏠 address: {address_div.get_text().strip()}")

                        phone_span = title_section.find('span', class_='phone')
                        if phone_span:
                            print(f"        📞 phone: {phone_span.get_text().strip()}")
                else:
                    print(f"      ❌ No .siteInfo div found")

                # Check for data attributes
                if first_element.get('data-latitude'):
                    print(f"      🌍 Latitude: {first_element.get('data-latitude')}")
                if first_element.get('data-longitude'):
                    print(f"      🌍 Longitude: {first_element.get('data-longitude')}")

                print()  # Blank line between elements

        # Check if the page structure matches our expectations
        print("\n🔍 Checking page structure...")

        categories = soup.find('ul', id='categories')
        if categories:
            print(f"  ✅ Found #categories container")
            parishes_category = categories.find('li', class_='category')
            if parishes_category:
                category_name = parishes_category.find('div', class_='categoryName')
                if category_name:
                    print(f"    📂 Category: {category_name.get_text().strip()}")

                sites_list = parishes_category.find('ul', class_='sites')
                if sites_list:
                    sites = sites_list.find_all('li', class_='site')
                    print(f"    🏗️ Found {len(sites)} sites in category")
                else:
                    print(f"    ❌ No ul.sites found")
            else:
                print(f"    ❌ No .category found")
        else:
            print(f"  ❌ No #categories found")

        # Check for JavaScript-loaded content
        print("\n🔍 Checking for dynamic content...")

        # Try to wait for content to load and check again
        time.sleep(3)

        # Check if clicking on categories makes parishes appear
        try:
            parishes_category = driver.find_element(By.CSS_SELECTOR, "li.category")
            if parishes_category:
                print(f"  📂 Found parishes category, checking if clickable...")
                parishes_category.click()
                time.sleep(2)

                # Re-check for parish elements after clicking
                updated_soup = BeautifulSoup(driver.page_source, 'html.parser')
                updated_sites = updated_soup.find_all('li', class_='site')
                print(f"  🔄 After clicking: {len(updated_sites)} sites found")

        except Exception as e:
            print(f"  ⚠️ Could not interact with category: {str(e)[:50]}...")

        # Check the current page source for any parishes data
        print(f"\n📊 Page statistics:")
        print(f"  📄 Total HTML length: {len(html_content):,} characters")
        print(f"  🔍 Contains 'parish': {'parish' in html_content.lower()}")
        print(f"  🔍 Contains 'church': {'church' in html_content.lower()}")
        print(f"  🔍 Contains 'site': {'site' in html_content.lower()}")
        print(f"  🔍 Contains 'siteInfo': {'siteInfo' in html_content}")

    except Exception as e:
        print(f"❌ Debug error: {e}")

    finally:
        driver.quit()
        print("\n🧹 Debug WebDriver closed")

# Uncomment the line below to run the debug function
# debug_parish_finder_structure()

print("✅ Debug function loaded - uncomment the last line to run debugging")

✅ Debug function loaded - uncomment the last line to run debugging


In [ ]:
# =============================================================================
# CELL 8.6: Debug and Fix Parish Finder Extraction (ADD THIS TO DEBUG THE ISSUE)
# =============================================================================

def debug_and_fix_parish_finder():
    """Debug the Parish Finder extraction and show what's happening"""

    print(f"🔍 DEBUGGING PARISH FINDER EXTRACTION ISSUE")
    print(f"{'='*60}")

    driver = setup_enhanced_driver()

    try:
        # Load the Parma parish finder page
        url = "https://www.parma.org/parishfinder"
        print(f"📥 Loading: {url}")
        driver.get(url)
        time.sleep(5)

        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')

        # Check the structure
        print(f"\n🔍 Analyzing page structure...")

        # Look for categories
        categories = soup.find('ul', id='categories')
        if categories:
            print(f"✅ Found #categories container")

            # Look for parish category
            parishes_li = categories.find('li', class_='category')
            if parishes_li:
                category_name_div = parishes_li.find('div', class_='categoryName')
                if category_name_div:
                    category_name = category_name_div.get_text().strip()
                    print(f"📂 Category name: '{category_name}'")

                # Look for sites within this category
                sites_ul = parishes_li.find('ul', class_='sites')
                if sites_ul:
                    sites = sites_ul.find_all('li', class_='site')
                    print(f"🏗️ Found {len(sites)} sites in parishes category")

                    # Analyze first few sites
                    for i, site in enumerate(sites[:5], 1):
                        print(f"\n  📍 Site {i}:")
                        print(f"      ID: {site.get('id', 'No ID')}")
                        print(f"      Data-latitude: {site.get('data-latitude', 'None')}")
                        print(f"      Data-longitude: {site.get('data-longitude', 'None')}")

                        # Look for name
                        name_div = site.find('div', class_='name')
                        if name_div:
                            name = name_div.get_text().strip()
                            print(f"      Name: '{name}'")
                        else:
                            print(f"      ❌ No name div found")

                        # Look for city
                        city_div = site.find('div', class_='city')
                        if city_div:
                            city = city_div.get_text().strip()
                            print(f"      City: '{city}'")
                        else:
                            print(f"      ❌ No city div found")

                        # Look for siteInfo
                        site_info = site.find('div', class_='siteInfo')
                        if site_info:
                            print(f"      ✅ Has siteInfo section")

                            # Check for detailed data within siteInfo
                            main_section = site_info.find('div', class_='main')
                            if main_section:
                                title_section = main_section.find('div', class_='title')
                                if title_section:
                                    # Check for address
                                    address_div = title_section.find('div', class_='address')
                                    if address_div:
                                        address = address_div.get_text().strip()
                                        print(f"        📍 Address: '{address}'")

                                    # Check for phone
                                    phone_holder = title_section.find('div', class_='phoneFaxHolder')
                                    if phone_holder:
                                        phone_span = phone_holder.find('span', class_='phone')
                                        if phone_span:
                                            phone = phone_span.get_text().strip()
                                            print(f"        📞 Phone: '{phone}'")
                        else:
                            print(f"      ❌ No siteInfo section")
                else:
                    print(f"❌ No ul.sites found in parishes category")
            else:
                print(f"❌ No li.category found in categories")
        else:
            print(f"❌ No #categories container found")

        # Now test the actual extraction
        print(f"\n🔧 Testing actual extraction logic...")

        # Create pattern detector
        detector = PatternDetector()
        pattern = detector.detect_pattern(html_content, url)
        print(f"📊 Detected pattern: {pattern.listing_type.value}")

        # Create extractor
        extractor = ParishFinderExtractor(pattern)

        # Test extraction
        parishes = extractor.extract(driver, soup, url)
        print(f"\n📊 Extraction results: {len(parishes)} parishes")

        # Show details of extracted parishes
        for i, parish in enumerate(parishes, 1):
            print(f"  {i}. {parish.name}")
            print(f"     City: {parish.city}")
            print(f"     Address: {parish.address}")
            print(f"     Phone: {parish.phone}")
            print(f"     Website: {parish.website}")
            print(f"     Coordinates: {parish.latitude}, {parish.longitude}")
            print(f"     Method: {parish.extraction_method}")
            print(f"     Confidence: {parish.confidence_score}")
            print()

        return parishes

    except Exception as e:
        print(f"❌ Debug error: {e}")
        return []

    finally:
        driver.quit()
        print("🧹 Debug WebDriver closed")

# Run the debug function
debug_results = debug_and_fix_parish_finder()

print("✅ Debug function completed")

🔍 DEBUGGING PARISH FINDER EXTRACTION ISSUE
🔧 Setting up enhanced Chrome WebDriver...
✅ Chrome WebDriver initialized successfully
📥 Loading: https://www.parma.org/parishfinder

🔍 Analyzing page structure...
✅ Found #categories container
📂 Category name: 'Parishes'
🏗️ Found 31 sites in parishes category

  📍 Site 1:
      ID: 20885
      Data-latitude: 41.6233383
      Data-longitude: -87.91294529999999
      Name: 'Annunciation of the Mother of God'
      City: 'Homer Glen'
      ✅ Has siteInfo section
        📍 Address: '14610 S. Will Cook Road, Homer Glen, Illinois, 60491-9212'
        📞 Phone: '708-645-0241'

  📍 Site 2:
      ID: 36949
      Data-latitude: 0.0
      Data-longitude: 0.0
      Name: 'Byzantine Catholic Outreach of Cincinnati'
      City: 'Cincinnati'
      ✅ Has siteInfo section
        📍 Address: 'St Anthony of Padua, Maronite Catholic Church,  Victory Pkwy, Cincinnati, OH 45206, Cincinnati, OH, 45206'

  📍 Site 3:
      ID: 20893
      Data-latitude: 41.439759
     

In [ ]:
# =============================================================================
# CELL 8.7: Test Fixed Parish Finder Extractor (QUICK TEST)
# =============================================================================

def test_fixed_parish_finder():
    """Quick test of the fixed Parish Finder extractor"""

    print(f"🧪 TESTING FIXED PARISH FINDER EXTRACTOR")
    print(f"{'='*50}")

    driver = setup_enhanced_driver()

    try:
        # Load the page
        url = "https://www.parma.org/parishfinder"
        print(f"📥 Loading: {url}")
        driver.get(url)
        time.sleep(5)

        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')

        # Test pattern detection
        detector = PatternDetector()
        pattern = detector.detect_pattern(html_content, url)
        print(f"📊 Pattern: {pattern.listing_type.value} (confidence: {pattern.confidence_score})")

        # Test extraction
        extractor = FixedParishFinderExtractor(pattern)
        parishes = extractor.extract(driver, soup, url)

        print(f"\\n📊 RESULTS: Extracted {len(parishes)} parishes")

        if parishes:
            print(f"\\n📋 First 5 parishes extracted:")
            for i, parish in enumerate(parishes[:5], 1):
                print(f"  {i}. {parish.name}")
                if parish.city:
                    print(f"     📍 City: {parish.city}")
                if parish.address or parish.full_address:
                    addr = parish.address or parish.full_address
                    print(f"     🏠 Address: {addr[:50]}...")
                if parish.phone:
                    print(f"     📞 Phone: {parish.phone}")
                if parish.website:
                    print(f"     🌐 Website: {parish.website}")
                if parish.latitude and parish.longitude:
                    print(f"     🌍 Coordinates: {parish.latitude}, {parish.longitude}")
                if parish.clergy_info:
                    print(f"     👥 Clergy: {parish.clergy_info[:50]}...")
                print(f"     🎯 Confidence: {parish.confidence_score}, Method: {parish.extraction_method}")
                print()

            if len(parishes) > 5:
                print(f"  ... and {len(parishes) - 5} more parishes")

            # Show quality statistics
            has_address = sum(1 for p in parishes if p.address or p.full_address)
            has_phone = sum(1 for p in parishes if p.phone)
            has_website = sum(1 for p in parishes if p.website)
            has_coordinates = sum(1 for p in parishes if p.latitude and p.longitude)
            has_clergy = sum(1 for p in parishes if p.clergy_info)

            print(f"\\n📊 Data Quality Summary:")
            print(f"  📍 Addresses: {has_address}/{len(parishes)} ({has_address/len(parishes)*100:.1f}%)")
            print(f"  📞 Phones: {has_phone}/{len(parishes)} ({has_phone/len(parishes)*100:.1f}%)")
            print(f"  🌐 Websites: {has_website}/{len(parishes)} ({has_website/len(parishes)*100:.1f}%)")
            print(f"  🌍 Coordinates: {has_coordinates}/{len(parishes)} ({has_coordinates/len(parishes)*100:.1f}%)")
            print(f"  👥 Clergy Info: {has_clergy}/{len(parishes)} ({has_clergy/len(parishes)*100:.1f}%)")

            return parishes
        else:
            print("❌ No parishes extracted - there may still be an issue")
            return []

    except Exception as e:
        print(f"❌ Test error: {e}")
        return []

    finally:
        driver.quit()
        print("\\n🧹 Test WebDriver closed")

# Run the test
print("🧪 Running test of fixed extractor...")
test_results = test_fixed_parish_finder()

if test_results and len(test_results) > 20:
    print(f"\\n🎉 SUCCESS! Fixed extractor found {len(test_results)} parishes")
    print(f"📊 This should resolve the issue where only 1 parish was being saved to the database")
else:
    print(f"\\n⚠️ Still having issues - may need further debugging")

print("✅ Test completed")

🧪 Running test of fixed extractor...
🧪 TESTING FIXED PARISH FINDER EXTRACTOR
🔧 Setting up enhanced Chrome WebDriver...
✅ Chrome WebDriver initialized successfully
📥 Loading: https://www.parma.org/parishfinder
📊 Pattern: parish_finder (confidence: 0.95)
❌ Test error: name 'FixedParishFinderExtractor' is not defined
\n🧹 Test WebDriver closed
\n⚠️ Still having issues - may need further debugging
✅ Test completed


In [ ]:
    # =============================================================================
    # CELL 9: Display Results and Analysis (UPDATED)
    # =============================================================================

    print(f"\\n{'='*70}")
    print(f"📊 ENHANCED EXTRACTION SUMMARY")
    print(f"{'='*70}")
    print(f"Total dioceses processed: {summary_stats['total_dioceses']}")
    print(f"Successful extractions: {summary_stats['successful_extractions']}")
    print(f"Success rate: {summary_stats.get('success_rate', 0):.1f}%")
    print(f"Total parishes found: {summary_stats['total_parishes_found']}")

    if summary_stats['total_dioceses'] > 0:
        print(f"\n{'='*70}")
        print(f"📊 ENHANCED EXTRACTION SUMMARY")
        print(f"{'='*70}")
        print(f"Total dioceses processed: {summary_stats['total_dioceses']}")
        print(f"Successful extractions: {summary_stats['successful_extractions']}")
        print(f"Success rate: {summary_stats.get('success_rate', 0):.1f}%")
        print(f"Total parishes found: {summary_stats['total_parishes_found']}")
        if summary_stats['successful_extractions'] > 0:
            print(f"Average parishes per diocese: {summary_stats.get('avg_parishes_per_diocese', 0):.1f}")
        print(f"\n\n📈 Pattern Distribution:")
        for pattern, count in summary_stats['pattern_distribution'].items():
            percentage = (count / summary_stats['total_dioceses']) * 100
            print(f"  {pattern.replace('_', ' ').title()}: {count} dioceses ({percentage:.1f}%)")
        print(f"\n\n🔧 Extraction Method Usage:")
        for method, count in summary_stats['extraction_method_usage'].items():
            print(f"  {method}: {count} times")
        print(f"\n\n🔍 Detailed Results:")
        for result in all_results:
            status = "✅" if result['success'] else "❌"
            parish_count = len(result['parishes_found'])
            pattern_info = ""
            if result['pattern_detected']:
                pattern_info = f" [{result['pattern_detected']['platform']} / {result['pattern_detected']['listing_type']}]"
            print(f"  {status} {result['diocese_name']}: {parish_count} parishes{pattern_info}")
            print(f"      Main URL: {result['diocese_url']}")
            print(f"      Parish Directory: {result['parish_directory_url']}")
            if result['extraction_methods_used']:
                methods = ', '.join(result['extraction_methods_used'])
                print(f"      Methods: {methods}")
            if result['errors']:
                for error in result['errors']:
                    print(f"      Error: {error[:100]}...")
            print()  # Add blank line between dioceses

        # Save summary to file for analysis
        summary_filename = f"extraction_summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(summary_filename, 'w') as f:
            json.dump({
                'summary_stats': summary_stats,
                'detailed_results': [
                    [result, result['errors']] if result.get('errors') and len(result['errors']) > 0 else result
                    for result in all_results
                ]
            }, f, indent=2, default=str)
        print(f"💾 Detailed results saved to: {summary_filename}")

    else:
        print("❌ No dioceses with parish directory URLs found to process")

    print(f"\n🎉 Enhanced pattern-based extraction complete!")
    print(f"{'='*70}")

\n======================================================================
📊 ENHANCED EXTRACTION SUMMARY
Total dioceses processed: 1
Successful extractions: 1
Success rate: 100.0%
Total parishes found: 24

📊 ENHANCED EXTRACTION SUMMARY
Total dioceses processed: 1
Successful extractions: 1
Success rate: 100.0%
Total parishes found: 24
Average parishes per diocese: 24.0


📈 Pattern Distribution:
  Ecatholic Parish Finder: 1 dioceses (100.0%)


🔧 Extraction Method Usage:
  ParishFinderExtractor: 1 times


🔍 Detailed Results:
  ✅ Eparchy of Parma: 24 parishes [ecatholic / parish_finder]
      Main URL: http://www.parma.org/
      Parish Directory: https://www.parma.org/parishfinder
      Methods: ParishFinderExtractor

💾 Detailed results saved to: extraction_summary_20250529_013222.json

🎉 Enhanced pattern-based extraction complete!
